In [2]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    print("Installing IDAES, GLPK, IPOPT using helper script...")
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()
    print("Helper script finished.")

    # Añadir la instalación de HiGHS aquí
    print("Installing HiGHS...")
    !pip install highspy
    print("HiGHS installed.")
    print("instalando folium")
    !pip install folium
    print("folium instalado")

--2025-05-28 04:42:13--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-28 04:42:13 (59.2 MB/s) - ‘helper.py’ saved [7171/7171]

Installing IDAES, GLPK, IPOPT using helper script...
Installing idaes via pip...
idaes was successfully installed
Installing glpk via apt-get...
Running idaes get-extensions to install Ipopt, k_aug, and more...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)

Couenne 0.5.8 -- an Open-Source solver for Mixe

# Caso Base

In [ ]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

# --- 1. Data Generation (Simulating CSV files) ---
def generate_sample_data(num_depots, num_clients, num_vehicles):
    """Generates sample dataframes for depots, clients, and vehicles."""
    depots_data = {
        'DepotID': [f'D{i+1}' for i in range(num_depots)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_depots)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_depots)],
        'DepotCapacity': [random.randint(500, 1000) for _ in range(num_depots)]
    }
    depots_df = pd.DataFrame(depots_data)

    clients_data = {
        'ClientID': [f'C{i+1}' for i in range(num_clients)],
        'Demand': [random.randint(10, 50) for _ in range(num_clients)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_clients)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_clients)]
    }
    clients_df = pd.DataFrame(clients_data)

    vehicles_data = {
        'VehicleID': [f'V{i+1}' for i in range(num_vehicles)],
        'Capacity': [random.randint(80, 150) for _ in range(num_vehicles)],
        'Range': [random.randint(100, 300) for _ in range(num_vehicles)] # Range in km
    }
    vehicles_df = pd.DataFrame(vehicles_data)

    depots_df.to_csv('depots.csv', index=False)
    clients_df.to_csv('clients.csv', index=False)
    vehicles_df.to_csv('vehicles.csv', index=False)
    print("Generated sample data saved to CSVs (depots.csv, clients.csv, vehicles.csv).")

    return depots_df, clients_df, vehicles_df


# --- Función para obtener matriz de distancias/tiempos de la API ---
def get_matrix_from_ors_api(locations_dict, node_ids, api_key):
    """
    Obtiene la matriz de distancias y duraciones de la API de Openrouteservice.
    locations_dict: dict {node_id: (lon, lat)}
    node_ids: lista ordenada de node_ids (Depots + Clients)
    """
    api_locations = [[locations_dict[node_id][0], locations_dict[node_id][1]] for node_id in node_ids]

    body = {"locations": api_locations, "metrics": ["distance", "duration"]}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    print(f"Calling Openrouteservice API for matrix with {len(node_ids)} locations...")
    print(f"Requesting metrics: distance, duration.")

    try:
        response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)

        if response.status_code == 200:
            data = response.json()
            distance_matrix_meters = data.get("distances")
            duration_matrix_seconds = data.get("durations")

            if distance_matrix_meters and duration_matrix_seconds:
                print("API call successful. Matrix received.")
                distances_km = {}
                durations_sec = {}
                n = len(node_ids)
                for i in range(n):
                    for j in range(n):
                        from_node = node_ids[i]
                        to_node = node_ids[j]
                        dist_m = distance_matrix_meters[i][j] if distance_matrix_meters[i] and distance_matrix_meters[i][j] is not None else float('inf')
                        dur_s = duration_matrix_seconds[i][j] if duration_matrix_seconds[i] and duration_matrix_seconds[i][j] is not None else float('inf')

                        distances_km[(from_node, to_node)] = dist_m / 1000.0 # Convertir metros a km
                        durations_sec[(from_node, to_node)] = dur_s # Segundos

                return distances_km, durations_sec # Retornamos ambas matrices

            else:
                print("API response missing 'distances' or 'durations' data.")
                print(f"Response body: {response.text}")
                return None, None

        else:
            print(f"API call failed with status code {response.status_code}: {response.reason}")
            print(f"Response body: {response.text}")
            return None, None

    except requests.exceptions.Timeout:
        print(f"API request timed out after 60 seconds.")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        return None, None
    except json.JSONDecodeError:
        print(f"API response is not valid JSON. Response body: {response.text}")
        return None, None


# --- 2. Preprocessing and Parameter Calculation (Modificada) ---
def preprocess_data(depots_df, clients_df, vehicles_df, api_key):
    """Processes raw dataframes into dictionaries for Pyomo."""
    params = {}

    locations = {}
    for _, row in depots_df.iterrows():
        locations[row['DepotID']] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations[row['ClientID']] = (row['Longitude'], row['Latitude'])

    params['locations'] = locations

    all_nodes = depots_df['DepotID'].tolist() + clients_df['ClientID'].tolist()

    # Llama a la API para obtener las matrices de distancia y duración
    distances_km, durations_sec = get_matrix_from_ors_api(locations, all_nodes, api_key)

    if distances_km is None:
        print("Error getting distance/duration matrix from API. Cannot proceed.")
        return None

    params['distances'] = distances_km
    params['durations'] = durations_sec # Guardamos también las duraciones

    params['depot_capacity'] = dict(zip(depots_df['DepotID'], depots_df['DepotCapacity']))
    params['client_demand'] = dict(zip(clients_df['ClientID'], clients_df['Demand']))
    params['vehicle_capacity'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity']))
    params['vehicle_range'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Range']))

    params['F_freight_cost'] = 5000
    params['M_maintenance_cost'] = 23
    params['Pf_fuel_cost'] = 525.1
    params['total_unit_distance_cost'] = params['F_freight_cost'] + params['M_maintenance_cost'] + params['Pf_fuel_cost']

    params['set_D'] = depots_df['DepotID'].tolist()
    params['set_C'] = clients_df['ClientID'].tolist()
    params['set_V'] = vehicles_df['VehicleID'].tolist()
    params['set_N'] = params['set_D'] + params['set_C']

    return params

# --- 3. Pyomo Model Implementation ---
def build_vrp_model(params):
    """Builds the Pyomo VRP model."""
    model = pyo.ConcreteModel(name="VehicleRoutingProblem")

    # --- SETS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])
    model.C_plus_D = model.C | model.D

    # --- PARAMETERS ---
    model.dist = pyo.Param(model.N, model.N, initialize=params['distances'])
    # model.duration = pyo.Param(model.N, model.N, initialize=params['durations']) # Descomentar si necesitas duraciones en las restricciones

    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.cost_per_km = pyo.Param(initialize=params['total_unit_distance_cost'])
    model.num_clients = pyo.Param(initialize=len(params['set_C']))

    # --- DECISION VARIABLES ---
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary, rule=lambda m, c1, c2, k: m.x_c1c2k[c1,c2,k].setlb(0) if c1==c2 else None )
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)
    model.u_jk = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals)

    # --- OBJECTIVE FUNCTION (Minimize total distance cost) ---
    def objective_rule(m):
        cost = 0
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D for k in m.V)
        return cost
    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- CONSTRAINTS ---
    # 1. Client Assignment: Each client j is assigned to exactly one vehicle k.
    def client_assignment_rule(m, j):
        return sum(m.Y_jk[j,k] for k in m.V) == 1
    model.client_assignment_con = pyo.Constraint(model.C, rule=client_assignment_rule)

    # 2. Vehicle Capacity: Total demand of clients assigned to vehicle k must not exceed its capacity.
    def vehicle_capacity_rule(m, k):
        return sum(m.client_demand[j] * m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    model.vehicle_capacity_con = pyo.Constraint(model.V, rule=vehicle_capacity_rule)

    # 3a: A vehicle starts from at most one depot to at most one client.
    def vehicle_must_start_if_used_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) <= 1
    model.vehicle_start_at_most_one_depot_con = pyo.Constraint(model.V, rule=vehicle_must_start_if_used_rule)

    # 3b: If a vehicle is used (serves any client), it must start from a depot.
    def link_Y_to_z_start_rule(m, k):
         M = m.num_clients
         return sum(m.Y_jk[j,k] for j in m.C) <= M * sum(m.z_djk[d,j,k] for d in m.D for j in m.C)
    model.link_Y_to_z_start_con = pyo.Constraint(model.V, rule=link_Y_to_z_start_rule)

    # 4. Route End: If a vehicle starts a tour, it must end at a depot.
    def vehicle_tour_end_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) == sum(m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
    model.vehicle_tour_end_con = pyo.Constraint(model.V, rule=vehicle_tour_end_rule)

    # 5. Flow Conservation at Client Nodes for vehicle k:
    def flow_conservation_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        outgoing_from_j = sum(m.x_prime_cdk[j_node,d,k] for d in m.D) + sum(m.x_c1c2k[j_node,c2,k] for c2 in m.C if j_node != c2)
        return incoming_to_j == outgoing_from_j
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_conservation_rule)

    # 6. Linking Y_jk (Assignment) to actual visit for vehicle k:
    def link_assign_to_visit_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        return incoming_to_j == m.Y_jk[j_node,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_to_visit_rule)

    # 7. Depot Capacity: Total demand leaving depot d as first stop.
    def depot_capacity_rule_pdf(m, d_node):
        return sum(m.client_demand[j] * m.z_djk[d_node,j,k] for j in m.C for k in m.V) <= m.depot_cap[d_node]
    model.depot_capacity_pdf_con = pyo.Constraint(model.D, rule=depot_capacity_rule_pdf)

    # 8. Vehicle Range (Uses real distances from model.dist)
    def vehicle_range_rule(m,k):
        dist_k = 0
        dist_k += sum(m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C)
        dist_k += sum(m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2)
        dist_k += sum(m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return dist_k <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 9. No self-loops for x_c1c2k (client to same client)
    def no_self_loops_clients_rule(m, c, k):
        return m.x_c1c2k[c,c,k] == 0
    model.no_self_loops_clients_con = pyo.Constraint(model.C, model.V, rule=no_self_loops_clients_rule)

    # 10. Subtour Elimination (MTZ Constraints)
    def u_bounds_lower_rule(m, j, k):
        return m.u_jk[j,k] >= 1.0 * m.Y_jk[j,k]
    model.u_bounds_lower_con = pyo.Constraint(model.C, model.V, rule=u_bounds_lower_rule)

    def u_bounds_upper_rule(m, j, k):
        return m.u_jk[j,k] <= (m.num_clients) * m.Y_jk[j,k]
    model.u_bounds_upper_con = pyo.Constraint(model.C, model.V, rule=u_bounds_upper_rule)

    def mtz_rule(m, c1, c2, k):
        if c1 == c2:
            return pyo.Constraint.Skip
        return m.u_jk[c1,k] - m.u_jk[c2,k] + (m.num_clients) * m.x_c1c2k[c1,c2,k] <= (m.num_clients) - 1.0
    model.mtz_con = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

    def mtz_initial_u_rule(m, d, j, k):
        return m.u_jk[j,k] <= 1.0 + (m.num_clients) * (1 - m.z_djk[d,j,k])
    model.mtz_initial_u_con = pyo.Constraint(model.D, model.C, model.V, rule=mtz_initial_u_rule)

    return model

# --- 4. Solving the Model and Extracting Results ---
def solve_model(model, solver_name='cbc', time_limit_seconds=None):
    """Solves the Pyomo model and returns results."""
    solver = SolverFactory(solver_name)

    if time_limit_seconds is not None:
        print(f"Setting time limit for solver '{solver_name}' to {time_limit_seconds} seconds.")
        if solver_name.lower() == 'gurobi':
            solver.options['TimeLimit'] = time_limit_seconds
        elif solver_name.lower() == 'cbc':
             solver.options['sec'] = time_limit_seconds
        elif solver_name.lower() == 'highs':
            solver.options['time_limit'] = time_limit_seconds
        else:
            print(f"Warning: Time limit option not known for solver '{solver_name}'. Time limit not set.")

    print(f"Attempting to solve with {solver_name}...")
    results = solver.solve(model, tee=True)

    return results

def extract_routes_and_details(model, results, params):
    """Extracts routes, loads, and costs from the solved model."""
    if not (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) or \
       not (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):
        print(f"Solver did not find an acceptable solution. Status: {results.solver.status}, Condition: {results.solver.termination_condition}")
        return None

    print("\n--- Solution Found ---")
    try:
        objective_value = pyo.value(model.total_cost)
        print(f"Total Cost: {objective_value:.2f} COP")
        solution_details = {'total_cost': objective_value, 'routes': {}}
    except Exception as e:
        print(f"Warning: Could not access objective value from model after solve: {e}.")
        print("Attempting to extract routes based on variable values...")
        solution_details = {'total_cost': float('inf'), 'routes': {}}


    for k_id in params['set_V']:
        vehicle_route = []
        vehicle_load = 0 # Calculated at the end
        vehicle_dist = 0
        current_node = None
        is_vehicle_used_for_route = False

        # Check start from depot (z_djk)
        for d_id in params['set_D']:
            for j_id in params['set_C']:
                 if pyo.value(model.z_djk[d_id,j_id,k_id]) is not None and pyo.value(model.z_djk[d_id,j_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    vehicle_route.append(j_id)
                    current_node = j_id
                    if (d_id, j_id) in params['distances']:
                         vehicle_dist += params['distances'][(d_id,j_id)]
                    else:
                         print(f"Warning: Distance not found for ({d_id}, {j_id}).")

                    is_vehicle_used_for_route = True
                    break
            if current_node: break

        if not is_vehicle_used_for_route:
            solution_details['routes'][k_id] = {'route': [], 'load': 0, 'distance': 0, 'clients_served': []}
            continue

        # Follow path between clients (x_c1c2k)
        clients_assigned_to_k = [j_id for j_id in params['set_C']
                                 if pyo.value(model.Y_jk[j_id,k_id]) is not None
                                 and pyo.value(model.Y_jk[j_id,k_id]) > 0.5]


        safety_counter = 0
        max_safety_steps = len(params['set_C']) * 2

        while current_node and current_node in params['set_C'] and safety_counter < max_safety_steps :

            found_next_client = False
            next_node = None

            for c2_id in params['set_C']:
                if c2_id != current_node:
                     if pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) is not None and pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) > 0.5:

                         if c2_id in vehicle_route:
                             print(f"Warning: Client {c2_id} would be revisited by {k_id} in current trace. Route: {vehicle_route}. Stopping trace for this vehicle.")
                             current_node = None
                             found_next_client = False
                             break

                         next_node = c2_id
                         found_next_client = True
                         break

            if found_next_client and next_node:
                 vehicle_route.append(next_node)
                 if (current_node, next_node) in params['distances']:
                     vehicle_dist += params['distances'][(current_node, next_node)]
                 else:
                     print(f"Warning: Distance not found for ({current_node}, {next_node}).")

                 current_node = next_node
                 safety_counter += 1

            else:
                break

        # Check for return to depot (x_prime_cdk) from the last client visited
        returned_to_depot = False
        last_node_in_trace = vehicle_route[-1] if vehicle_route else None

        if last_node_in_trace and last_node_in_trace in params['set_C']:
             for d_id in params['set_D']:
                 if pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) is not None and pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    if (last_node_in_trace, d_id) in params['distances']:
                         vehicle_dist += params['distances'][(last_node_in_trace,d_id)]
                    else:
                         print(f"Warning: Distance not found for ({last_node_in_trace}, {d_id}).")

                    returned_to_depot = True
                    break

        # Final load calculation based on Y_jk (clients assigned to this vehicle)
        actual_load_for_vehicle = sum(params['client_demand'][c] for c in clients_assigned_to_k)

        # Store the extracted route details
        solution_details['routes'][k_id] = {
            'route': vehicle_route,
            'load': actual_load_for_vehicle,
            'distance': vehicle_dist,
            'clients_served': clients_assigned_to_k
        }

        # Add warnings if the extracted route seems incomplete or problematic
        if is_vehicle_used_for_route:
             clients_in_traced_route = [node for node in vehicle_route if node in params['set_C']]
             assigned_but_not_traced = [c for c in clients_assigned_to_k if c not in clients_in_traced_route]
             traced_but_not_assigned = [c for c in clients_in_traced_route if c not in clients_assigned_to_k]
             if assigned_but_not_traced:
                  print(f"Warning: Vehicle {k_id} was assigned clients {assigned_but_not_traced} but did not visit them in trace.")
             if traced_but_not_assigned:
                  print(f"Warning: Vehicle {k_id} visited clients {traced_but_not_assigned} in trace which are not assigned to it.")

             route_starts_at_depot = len(vehicle_route) > 0 and vehicle_route[0] in params['set_D']
             route_ends_at_depot = len(vehicle_route) > 0 and vehicle_route[-1] in params['set_D']

             if is_vehicle_used_for_route and not (route_starts_at_depot and route_ends_at_depot):
                 if len(vehicle_route) > 1:
                      print(f"Warning: Vehicle {k_id} was used but its traced route does not start and end at a depot. Route: {' -> '.join(vehicle_route)}")


    return solution_details

# --- 5. Visualization (NetworkX) ---
# CORREGIDA: Usar la clave correcta 'depot_capacity'
def visualize_routes(solution_details, params):
    """Visualizes the routes on a graph."""
    if not solution_details or not solution_details['routes']:
        print("No solution to visualize.")
        return

    loc_data = params['locations']
    G = nx.DiGraph()

    pos = {}
    node_colors = {}
    node_labels = {}
    node_size = {}

    for d_id in params['set_D']:
        G.add_node(d_id)
        pos[d_id] = (loc_data[d_id][0], loc_data[d_id][1])
        node_colors[d_id] = 'red'
        node_labels[d_id] = f"{d_id}\n(Cap:{params['depot_capacity'][d_id]})"
        node_size[d_id] = 3000

    for c_id in params['set_C']:
        G.add_node(c_id)
        pos[c_id] = (loc_data[c_id][0], loc_data[c_id][1])
        node_colors[c_id] = 'lightblue'
        node_labels[c_id] = f"{c_id}\n(D:{params['client_demand'][c_id]})"
        node_size[c_id] = 1500 + params['client_demand'][c_id] * 20


    plt.figure(figsize=(16, 12))
    nx.draw_networkx_nodes(G, pos,
                             node_color=[node_colors[n] for n in G.nodes()],
                             node_size=[node_size[n] for n in G.nodes()],
                             alpha=0.9, edgecolors='black', linewidths=1)

    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=7, font_weight='bold')


    edge_colors = ['b', 'g', 'c', 'm', 'y', 'k', '#ff7f0e', '#d62728']
    vehicle_styles = ['-', '--', '-.', ':']

    for node_id in params['set_D'] + params['set_C']:
        if node_id not in G.nodes():
            print(f"Warning: Node {node_id} missing from graph before drawing edges.")
            if node_id in loc_data:
                 pos[node_id] = (loc_data[node_id][0], loc_data[node_id][1])
                 G.add_node(node_id)

    print("\n--- Visualized Routes (NetworkX) ---")
    vehicle_idx = 0
    vehicles_with_routes = []

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            vehicles_with_routes.append(v_id)
            color = edge_colors[vehicle_idx % len(edge_colors)]
            style = vehicle_styles[vehicle_idx % len(vehicle_styles)]

            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]

                if u_node in G.nodes() and v_node in G.nodes():
                    G.add_edge(u_node, v_node, color=color, style=style, vehicle=v_id)
                else:
                    print(f"Warning: Skipping edge ({u_node} -> {v_node}) for Vehicle {v_id} as one or both nodes are not in the graph.")

            vehicle_idx += 1

    edge_list_with_attributes = []
    for u, v, data in G.edges(data=True):
        edge_list_with_attributes.append(((u, v), data.get('color', 'gray'), data.get('style', '-')))

    if edge_list_with_attributes:
         edges_to_draw = [edge[0] for edge in edge_list_with_attributes]
         colors_for_draw = [edge[1] for edge in edge_list_with_attributes]
         styles_for_draw = [edge[2] for edge in edge_list_with_attributes]

         pos_for_edges = {node: pos[node] for node in G.nodes() if node in pos}

         if pos_for_edges:
             nx.draw_networkx_edges(G, pos_for_edges,
                                    edgelist=edges_to_draw,
                                    edge_color=colors_for_draw,
                                    style=styles_for_draw,
                                    width=2.0,
                                    alpha=0.7,
                                    arrowstyle='->',
                                    arrows=True,
                                    node_size=[node_size.get(node, 1000) for node in pos_for_edges.keys()],
                                    arrowsize=20
                                    )
         else:
             print("Warning: No node positions available to draw edges.")

    handles = []
    labels = []
    vehicle_idx = 0

    for v_id in vehicles_with_routes:
         data = solution_details['routes'][v_id]
         color = edge_colors[vehicle_idx % len(edge_colors)]
         style = vehicle_styles[vehicle_idx % len(vehicle_styles)]
         handles.append(plt.Line2D([0], [0], color=color, lw=2, linestyle=style))
         labels.append(f"Veh {v_id}: Load {data['load']:.0f}/{params['vehicle_capacity'][v_id]}, Dist {data['distance']:.1f}/{params['vehicle_range'][v_id]:.1f} km")
         vehicle_idx += 1


    if handles:
        plt.legend(handles, labels, title="Vehicle Routes", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
        plt.subplots_adjust(right=0.7)


    plt.title("Vehicle Routes (from Openrouteservice API distances)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.show()


# --- New Function for Folium Visualization ---
def visualize_routes_folium(solution_details, params):
    """Visualizes the routes on an interactive Folium map."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution to visualize with Folium.")
        return None

    print("\nGenerating Folium map...")

    loc_data = params['locations'] # {node_id: (lon, lat)}

    # Determine map center - use the first depot's location as a simple center
    # Folium expects [latitude, longitude]
    map_center = [loc_data[params['set_D'][0]][1], loc_data[params['set_D'][0]][0]] if params['set_D'] else [4.6, -74.0] # Fallback for Bogotá

    # Create base map
    m = folium.Map(location=map_center, zoom_start=12) # Adjust zoom as needed

    # Add Depot Markers
    for d_id in params['set_D']:
        lat, lon = loc_data[d_id][1], loc_data[d_id][0] # Folium expects [lat, lon]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Depot {d_id}</b><br>Capacity: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home') # Use a home icon for depots
        ).add_to(m)

    # Add Client Markers
    for c_id in params['set_C']:
        lat, lon = loc_data[c_id][1], loc_data[c_id][0]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Client {c_id}</b><br>Demand: {params['client_demand'][c_id]}",
            icon=folium.Icon(color='blue', icon='info-sign') # Use info-sign for clients
        ).add_to(m)

    # Add Vehicle Routes as Polylines
    route_colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue', 'darkgreen', 'darkpurple']

    vehicle_idx = 0
    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            route_coords = []
            for node_id in route:
                 if node_id in loc_data:
                      route_coords.append([loc_data[node_id][1], loc_data[node_id][0]])
                 else:
                      print(f"Warning: Location data missing for node {node_id} in route for Vehicle {v_id}. Skipping polyline segment.")
                      pass


            if len(route_coords) > 1:
                color = route_colors[vehicle_idx % len(route_colors)]
                popup_text = f"<b>Vehicle {v_id}</b><br>"
                popup_text += f"Route: {' -> '.join(route)}<br>"
                popup_text += f"Load: {data['load']:.0f}/{params['vehicle_capacity'][v_id]}<br>"
                popup_text += f"Distance: {data['distance']:.2f} km"

                folium.PolyLine(
                    route_coords,
                    color=color,
                    weight=2.5,
                    opacity=0.8,
                    popup=popup_text
                ).add_to(m)

                vehicle_idx += 1

    print("Folium map generated.")
    return m

# --- New Function to Display Vehicle Summary ---
def display_vehicle_summary(solution_details, model_params):
    """Displays a summary table for each vehicle."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution details available to display vehicle summary.")
        return

    print("\n--- Vehicle Summary ---")

    summary_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available in model_params. Cannot calculate total time.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)
    if 'Pf_fuel_cost' not in model_params:
         print("Warning: 'Pf_fuel_cost' not found in model_params. Fuel cost will be 0.")


    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        demands_satisfied = initial_load

        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        summary_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': f"{initial_load:.0f}",
            'RouteSequence': route_sequence_str,
            'ClientsServed': ", ".join(clients_served) if clients_served else "None",
            'DemandsSatisfied': f"{demands_satisfied:.0f}",
            'TotalDistance (km)': f"{total_distance:.2f}",
            'TotalTime (s)': f"{total_time_sec:.0f}",
            'FuelCost (COP)': f"{calculated_fuel_cost:.2f}"
        })

    summary_df = pd.DataFrame(summary_list)

    if not summary_df.empty:
        print(summary_df.to_string(index=False))
    else:
        print("No vehicle summary data generated.")


# --- New Function to Save Verification File ---
def save_verification_file(solution_details, model_params, case_number):
    """Saves the vehicle summary to a verification CSV file."""
    if not solution_details or not solution_details['routes']:
        print(f"\nNo solution details available to save verification file for case {case_number}.")
        return

    print(f"\nGenerating verification file for case {case_number}...")

    verification_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available. Total time in verification file will be 0 or inaccurate.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served_list = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        clients_served_count = len(clients_served_list)

        # Calculate Demands Satisfied (list of demands in order of visit)
        demands_satisfied_list = []
        if route:
            for node_id in route:
                # Only include demands for client nodes that are actually in the route
                if node_id in model_params['set_C']:
                    demands_satisfied_list.append(str(model_params['client_demand'][node_id]))
        demands_satisfied_str = "-".join(demands_satisfied_list) if demands_satisfied_list else "None"

        # Calculate Total Time from the route sequence and durations matrix
        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    # print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        # Convert total time from seconds to minutes
        total_time_min = total_time_sec / 60.0

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        verification_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': int(initial_load), # Should be integer demand sum
            'RouteSequence': route_sequence_str,
            'ClientsServed': clients_served_count,
            'DemandsSatisfied': demands_satisfied_str,
            'TotalDistance': round(total_distance, 2), # Round to 2 decimal places
            'TotalTime': round(total_time_min, 1), # Round to 1 decimal place for minutes
            'FuelCost': round(calculated_fuel_cost, 2) # Round to 2 decimal places
        })

    verification_df = pd.DataFrame(verification_list)

    if not verification_df.empty:
        # Ensure columns are in the specified order
        csv_columns_order = [
            'VehicleId',
            'DepotId',
            'InitialLoad',
            'RouteSequence',
            'ClientsServed',
            'DemandsSatisfied',
            'TotalDistance',
            'TotalTime',
            'FuelCost'
        ]
        verification_df = verification_df[csv_columns_order]

        # Define the filename
        filename = f"verificacion_caso{case_number}.csv"

        # Save the DataFrame to CSV
        try:
            verification_df.to_csv(filename, index=False)
            print(f"Verification file saved successfully: {filename}")
        except Exception as e:
            print(f"Error saving verification file {filename}: {e}")

    else:
        print(f"No verification data generated for case {case_number}.")


# --- Main Execution ---
if __name__ == "__main__":
    print("Starting VRP Model with Openrouteservice API distances.")
    num_depots = 1
    num_clients = 24
    num_vehicles = 8
    print(f"Problem size: {num_depots} depots, {num_clients} clients, {num_vehicles} vehicles.")

    # Define the case number for the verification file
    current_case_number = 1 # You can change this for different cases

    depots_df, clients_df, vehicles_df = None, None, None
    try:
        # Check if CSVs exist, if not, sample data will be generated and saved
        depots_df = pd.read_csv('depots.csv')
        clients_df = pd.read_csv('clients.csv')
        vehicles_df = pd.read_csv('vehicles.csv')
        print("Data loaded from CSVs.")
    except FileNotFoundError:
        print("CSV files not found (depots.csv, clients.csv, vehicles.csv). Generating sample data instead.")
        depots_df, clients_df, vehicles_df = generate_sample_data(num_depots, num_clients, num_vehicles)

    if depots_df is None or clients_df is None or vehicles_df is None:
        print("Failed to load or generate data. Exiting.")
        exit()

    print("\nPreprocessing data (calling API for distances)...")
    model_params = preprocess_data(depots_df, clients_df, vehicles_df, ORS_API_KEY)

    if model_params is None:
        print("Preprocessing failed (API error?). Exiting.")
        exit()

    print("\nBuilding Pyomo model...")
    vrp_model = build_vrp_model(model_params)
    print("Model built.")

    print("\nSolving model (this might take time depending on size and solver)...")
    solver_name = 'highs'
    solve_time_limit = 900

    try:
        results = solve_model(vrp_model, solver_name=solver_name, time_limit_seconds=solve_time_limit)

        if (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) and \
           (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):

            print(f"\nSolver terminated with status: {results.solver.status} and condition: {results.solver.termination_condition}")

            if results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
                 print("Result: Solver reached time limit. Reporting the best feasible solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.optimal:
                 print("Result: Optimal solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
                 print("Result: Feasible solution found (optimality not proven).")

            try:
                 objective_value = pyo.value(vrp_model.total_cost)
                 print(f"\nBest feasible objective value found: {objective_value:.2f} COP")
            except Exception as e:
                 print(f"\nWarning: Could not access objective value from model after successful solve status: {e}")
                 print("Proceeding to extract routes based on variable values...")

            extracted_solution = extract_routes_and_details(vrp_model, results, model_params)

            if extracted_solution:
                # Visualize using NetworkX (your existing method) - Comentada
                #print("\nVisualizing routes (NetworkX)...")
                #visualize_routes(extracted_solution, model_params) # Ensure this function is corrected in your file

                # Visualize using Folium
                print("\nVisualizing routes (Folium)...")
                folium_map = visualize_routes_folium(extracted_solution, model_params)
                if folium_map:
                    display(folium_map)

                # Display vehicle summary table
                display_vehicle_summary(extracted_solution, model_params)

                # NEW: Save verification file
                save_verification_file(extracted_solution, model_params, case_number=current_case_number)


            else:
                print("No solution details extracted.")

        elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
            print("\nSolver terminated with condition: Infeasible.")
            print("Model is infeasible. Check constraints and data.")

        else:
            print(f"\nSolver terminated with unexpected status: {results.solver.status}, condition: {results.solver.termination_condition}")
            print("Solver did not find an acceptable solution.")
            print("Check solver output above for details.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during model setup, solving, or post-processing: {e}")
        print(f"Ensure solver '{solver_name}' is installed and accessible.")
        print("Verify data loading, preprocessing, and model building steps if error occurred early.")

Starting VRP Model with Openrouteservice API distances.
Problem size: 1 depots, 24 clients, 8 vehicles.
Data loaded from CSVs.

Preprocessing data (calling API for distances)...
Calling Openrouteservice API for matrix with 25 locations...
Requesting metrics: distance, duration.
API call successful. Matrix received.

Building Pyomo model...
Model built.

Solving model (this might take time depending on size and solver)...
Setting time limit for solver 'highs' to 900 seconds.
Attempting to solve with highs...
Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 5633 rows; 5376 cols; 34944 nonzeros; 5184 integer variables (5184 binary)
Coefficient ranges:
  Matrix [1e+00, 8e+01]
  Cost   [2e+04, 5e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 8e+02]
Presolving model
5441 rows, 5184 cols, 34752 nonzeros  0s
5411 rows, 5120 cols, 30012 nonzeros  2s

Solving MIP model with:
   5411 rows
   5120 cols (4928 binary, 0 integer, 0 implied int., 192 c


--- Vehicle Summary ---
VehicleId DepotId InitialLoad                             RouteSequence         ClientsServed DemandsSatisfied TotalDistance (km) TotalTime (s) FuelCost (COP)
       V1     N/A           0                                  Not Used                  None                0               0.00             0           0.00
       V2      D1         118  D1 -> C16 -> C5 -> C17 -> C2 -> C3 -> D1  C2, C3, C5, C16, C17              118             112.67         12038       59160.68
       V3      D1          62                     D1 -> C19 -> C8 -> D1               C8, C19               62              45.13          5416       23699.85
       V4      D1         109             D1 -> C24 -> C15 -> C14 -> D1         C14, C15, C24              109              99.84          9770       52427.13
       V5      D1         121      D1 -> C20 -> C10 -> C11 -> C13 -> D1    C10, C11, C13, C20              121             135.67         14264       71242.72
       V6      D1    

# CASO 2

In [5]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

def get_matrix_from_ors_api(locations, nodes, api_key):
    """Calls OpenRouteService Matrix API and returns distance and duration matrices."""
    coords = [locations[node] for node in nodes]
    body = {"locations": coords, "metrics": ["distance", "duration"], "units": "km"}
    headers = {'Authorization': api_key}
    response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)
    if response.status_code == 200:
        data = response.json()
        raw_dist = data['distances']
        raw_dur = data['durations']
    else:
        print(f"ORS API call failed with status code {response.status_code}: {response.text}")
        return None, None
    return raw_dist, raw_dur

def preprocess_data(depots_df, clients_df, vehicles_df, api_key):
    """
    Processes raw dataframes into dictionaries for Pyomo.
    Returns None on API failure.
    """
    # --- 1. Create unique node identifiers to avoid conflicts ---
    # Prefix depot IDs with 'D' and client IDs with 'C'
    depots = ['D' + str(row['DepotID']) for _, row in depots_df.iterrows()]
    clients = ['C' + str(row['ClientID']) for _, row in clients_df.iterrows()]
    vehicles = vehicles_df['VehicleID'].tolist()
    all_nodes = depots + clients

    print("Depots:", depots)
    print("Clients:", clients)
    print("All nodes:", all_nodes)

    # Create locations dictionary with new identifiers
    locations = {}
    for _, row in depots_df.iterrows():
        locations['D' + str(row['DepotID'])] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations['C' + str(row['ClientID'])] = (row['Longitude'], row['Latitude'])

    # --- 2. Call ORS API ---
    raw_dist, raw_dur = get_matrix_from_ors_api(locations, all_nodes, api_key)
    if raw_dist is None:
        print("Error getting distance/duration matrix from API.")
        return None

    # Build distance matrices with new identifiers
    dist_dc = {}; dur_dc = {}
    dist_cc = {}; dur_cc = {}
    dist_cd = {}; dur_cd = {}

    for i, d in enumerate(depots):
        for j, c in enumerate(clients):
            dist_dc[(d, c)] = raw_dist[i][j + len(depots)]
            dur_dc[(d, c)] = raw_dur[i][j + len(depots)]

    for i, c1 in enumerate(clients):
        for j, c2 in enumerate(clients):
            dist_cc[(c1, c2)] = raw_dist[i + len(depots)][j + len(depots)]
            dur_cc[(c1, c2)] = raw_dur[i + len(depots)][j + len(depots)]

    for i, c in enumerate(clients):
        for j, d in enumerate(depots):
            dist_cd[(c, d)] = raw_dist[i + len(depots)][j]
            dur_cd[(c, d)] = raw_dur[i + len(depots)][j]

    # Create unified distance dictionary
    all_distances = {}
    all_distances.update(dist_dc)
    all_distances.update(dist_cc)
    all_distances.update(dist_cd)

    # Create parameter dictionaries with new identifiers
    depot_capacity = {}
    for _, row in depots_df.iterrows():
        depot_capacity['D' + str(row['DepotID'])] = row['Capacity']

    client_demand = {}
    for _, row in clients_df.iterrows():
        client_demand['C' + str(row['ClientID'])] = row['Demand']

    params = {
        'locations': locations,
        'dist_dc': dist_dc, 'dist_cc': dist_cc, 'dist_cd': dist_cd,
        'dur_dc': dur_dc, 'dur_cc': dur_cc, 'dur_cd': dur_cd,
        'distances': all_distances,
        'depot_capacity': depot_capacity,
        'client_demand': client_demand,
        'vehicle_capacity': dict(zip(vehicles, vehicles_df['Capacity'])),
        'vehicle_range': dict(zip(vehicles, vehicles_df['Range'])),
        'F_freight_cost': 5000,
        'M_maintenance_cost': 23,
        'Pf_fuel_cost': 525.1,
    }
    params['total_unit_distance_cost'] = (
        params['F_freight_cost']
      + params['M_maintenance_cost']
      + params['Pf_fuel_cost']
    )
    params['set_D'] = depots
    params['set_C'] = clients
    params['set_V'] = vehicles
    params['set_N'] = all_nodes

    print(f"Preprocessing complete: depots={len(depots)}, clients={len(clients)}, vehicles={len(vehicles)}")
    return params

def build_vrp_model(params):
    """Builds the Pyomo VRP model."""
    model = pyo.ConcreteModel(name="VehicleRoutingProblem")

    # --- SETS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])

    # --- PARAMETERS ---
    # Create full distance matrix for all node pairs
    full_dist = {}
    for n1 in params['set_N']:
        for n2 in params['set_N']:
            if (n1, n2) in params['distances']:
                full_dist[(n1, n2)] = params['distances'][(n1, n2)]
            elif n1 == n2:
                full_dist[(n1, n2)] = 0
            else:
                # This shouldn't happen if preprocessing is correct
                full_dist[(n1, n2)] = 9999

    model.dist = pyo.Param(model.N, model.N, initialize=full_dist)
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.num_clients = len(params['set_C'])
    model.cost_per_km = params['total_unit_distance_cost']

    # --- VARIABLES ---
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary)
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)
    model.u_jk = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals)

    # --- OBJECTIVE FUNCTION ---
    def objective_rule(m):
        cost = 0
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k]
                    for d in m.D for j in m.C for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k]
                    for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k]
                    for c in m.C for d in m.D for k in m.V)
        return cost
    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- CONSTRAINTS ---
    # 1. Each client visited exactly once
    model.client_assignment_con = pyo.Constraint(
        model.C,
        rule=lambda m, j: sum(m.Y_jk[j,k] for k in m.V) == 1
    )

    # 2. Vehicle capacity
    model.vehicle_capacity_con = pyo.Constraint(
        model.V,
        rule=lambda m, k: sum(m.client_demand[j]*m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    )

    # 3. Flow conservation
    def flow_cons_rule(m, j, k):
        in_to_j = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j)
        out_from_j = sum(m.x_prime_cdk[j,d,k] for d in m.D) + sum(m.x_c1c2k[j,c2,k] for c2 in m.C if c2 != j)
        return in_to_j == out_from_j
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_cons_rule)

    # 4. Link assignment to visit
    def link_assign_visit_rule(m, j, k):
        in_to_j = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j)
        return in_to_j == m.Y_jk[j,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_visit_rule)

    # 5. Depot capacity
    model.depot_capacity_con = pyo.Constraint(
        model.D,
        rule=lambda m, d: sum(m.client_demand[j]*m.z_djk[d,j,k] for j in m.C for k in m.V) <= m.depot_cap[d]
    )

    # 6. Vehicle range
    def vehicle_range_rule(m, k):
        tot = sum(m.dist[d,j]*m.z_djk[d,j,k] for d in m.D for j in m.C)
        tot += sum(m.dist[c1,c2]*m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2)
        tot += sum(m.dist[c,d]*m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return tot <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 7. No self loops
    model.no_self_loops_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, c, k: m.x_c1c2k[c,c,k] == 0
    )

    # 8. MTZ subtour elimination
    model.u_bounds_lower_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, j, k: m.u_jk[j,k] >= m.Y_jk[j,k]
    )
    model.u_bounds_upper_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, j, k: m.u_jk[j,k] <= m.num_clients * m.Y_jk[j,k]
    )

    # 9. Si un vehículo es usado, debe regresar a algún depósito
    def return_to_depot_if_used_rule(m, k):
        clients_served = sum(m.Y_jk[j, k] for j in m.C)
        depot_returns = sum(m.x_prime_cdk[j, d, k] for j in m.C for d in m.D)
        return clients_served <= m.num_clients * depot_returns
    model.return_to_depot_con = pyo.Constraint(model.V, rule=return_to_depot_if_used_rule)

    # 10. MTZ subtour elimination constraint
    def mtz_rule(m, c1, c2, k):
        if c1 == c2:
            return pyo.Constraint.Skip
        return (m.u_jk[c1,k] - m.u_jk[c2,k] + m.num_clients * m.x_c1c2k[c1,c2,k] <= m.num_clients - 1)
    model.mtz_con = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

    return model

def extract_routes_and_details(model, results, params):
    """Extracts routes from solved model with complete debugging."""
    if results.solver.termination_condition not in {
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.feasible,
            pyo.TerminationCondition.maxTimeLimit}:
        print(f"Solver did not find solution. Status: {results.solver.status}")
        return None

    print("\n--- Solution Found ---")
    try:
        objective_value = pyo.value(model.total_cost)
        print(f"Total Cost: {objective_value:.2f} COP")
    except:
        objective_value = float('inf')

    solution_details = {'total_cost': objective_value, 'routes': {}}

    for k_id in params['set_V']:
        print(f"\n=== ANALYZING VEHICLE {k_id} ===")

        # Find which clients are served by this vehicle
        clients_served = []
        for c_id in params['set_C']:
            if pyo.value(model.Y_jk[c_id, k_id]) and pyo.value(model.Y_jk[c_id, k_id]) > 0.5:
                clients_served.append(c_id)

        print(f"Clients served by vehicle {k_id}: {clients_served}")

        if not clients_served:
            solution_details['routes'][k_id] = {
                'route': [], 'load': 0, 'distance': 0, 'clients_served': []
            }
            continue

        # Debug all active connections for this vehicle
        print(f"\n--- All active connections for vehicle {k_id} ---")

        # Depot to client connections
        depot_client_connections = []
        for d_id in params['set_D']:
            for c_id in params['set_C']:
                val = pyo.value(model.z_djk[d_id, c_id, k_id])
                if val and val > 0.5:
                    depot_client_connections.append((d_id, c_id))
                    print(f"Depot->Client: {d_id} -> {c_id}")

        # Client to client connections
        client_client_connections = []
        for c1_id in params['set_C']:
            for c2_id in params['set_C']:
                if c1_id != c2_id:
                    val = pyo.value(model.x_c1c2k[c1_id, c2_id, k_id])
                    if val and val > 0.5:
                        client_client_connections.append((c1_id, c2_id))
                        print(f"Client->Client: {c1_id} -> {c2_id}")

        # Client to depot connections
        client_depot_connections = []
        for c_id in params['set_C']:
            for d_id in params['set_D']:
                val = pyo.value(model.x_prime_cdk[c_id, d_id, k_id])
                if val and val > 0.5:
                    client_depot_connections.append((c_id, d_id))
                    print(f"Client->Depot: {c_id} -> {d_id}")

        # Build route using graph traversal approach
        if not depot_client_connections:
            print(f"No depot-client connections found for vehicle {k_id}")
            continue

        # Find the starting depot and first client
        start_depot, first_client = depot_client_connections[0]
        route = [start_depot, first_client]
        current_node = first_client

        print(f"Starting route: {start_depot} -> {first_client}")

        # Follow the path through client-client connections
        visited_clients = {first_client}
        while len(visited_clients) < len(clients_served):
            next_node = None
            for c1, c2 in client_client_connections:
                if c1 == current_node and c2 not in visited_clients:
                    next_node = c2
                    break

            if next_node:
                route.append(next_node)
                visited_clients.add(next_node)
                current_node = next_node
                print(f"Added to route: -> {next_node}")
            else:
                print(f"No more client connections from {current_node}")
                break

        # Find the return to depot from the last client
        end_depot = None
        for c_id, d_id in client_depot_connections:
            if c_id == current_node:
                end_depot = d_id
                route.append(end_depot)
                print(f"Return to depot: {current_node} -> {end_depot}")
                break

        if not end_depot:
            print(f"WARNING: No return to depot found from {current_node}")

        # Calculate metrics
        total_load = sum(params['client_demand'][c] for c in clients_served)
        total_distance = 0
        for i in range(len(route) - 1):
            if (route[i], route[i+1]) in params['distances']:
                distance_segment = params['distances'][(route[i], route[i+1])]
                total_distance += distance_segment
                print(f"Distance {route[i]} -> {route[i+1]}: {distance_segment:.2f}km")
            else:
                print(f"Warning: Distance not found for {route[i]} -> {route[i+1]}")

        solution_details['routes'][k_id] = {
            'route': route,
            'load': total_load,
            'distance': total_distance,
            'clients_served': clients_served
        }

        print(f"\nFINAL ROUTE for Vehicle {k_id}: {' -> '.join(str(n) for n in route)}")
        print(f"Load: {total_load}, Distance: {total_distance:.1f}km")

    return solution_details

def visualize_routes_folium(solution_details, params):
    """Creates interactive Folium map."""
    if not solution_details or not solution_details['routes']:
        print("No solution to visualize.")
        return None

    loc_data = params['locations']

    # Center map
    all_lats = [loc[1] for loc in loc_data.values()]
    all_lons = [loc[0] for loc in loc_data.values()]
    center_lat = sum(all_lats) / len(all_lats)
    center_lon = sum(all_lons) / len(all_lons)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add depot markers
    for d_id in params['set_D']:
        lon, lat = loc_data[d_id]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Depot {d_id}</b><br>Capacity: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home', prefix='fa')
        ).add_to(m)

    # Add client markers
    for c_id in params['set_C']:
        lon, lat = loc_data[c_id]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Client {c_id}</b><br>Demand: {params['client_demand'][c_id]}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # Add routes
    colors = ['darkblue', 'darkgreen', 'purple', 'orange', 'darkred', 'cadetblue']
    route_idx = 0

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) < 2:
            continue

        # Get coordinates for route
        coords = []
        for node in route:
            if node in loc_data:
                lon, lat = loc_data[node]
                coords.append([lat, lon])

        if len(coords) >= 2:
            color = colors[route_idx % len(colors)]

            # Add route line
            folium.PolyLine(
                coords,
                color=color,
                weight=4,
                opacity=0.8,
                popup=f"<b>Vehicle {v_id}</b><br>Route: {' -> '.join(str(n) for n in route)}<br>Load: {data['load']}<br>Distance: {data['distance']:.1f}km"
            ).add_to(m)

            # Add directional arrows
            for i in range(len(coords) - 1):
                mid_lat = (coords[i][0] + coords[i+1][0]) / 2
                mid_lon = (coords[i][1] + coords[i+1][1]) / 2
                folium.CircleMarker(
                    [mid_lat, mid_lon],
                    radius=3,
                    color=color,
                    fill=True,
                    popup=f"Vehicle {v_id}: {route[i]} → {route[i+1]}"
                ).add_to(m)

            route_idx += 1

    print(f"Generated map with {route_idx} routes")
    return m

def display_vehicle_summary(solution_details, params):
    """Display summary table and saves it to verificacion_caso2.csv.""" # Modificación en el docstring
    if not solution_details:
        print("No solution details to summarize.") # Mensaje modificado para claridad
        return

    print("\n--- Vehicle Summary ---")
    summary_data = []

    for v_id, data in solution_details['routes'].items():
        if not data['route']: # [cite: 125]
            continue

        route_str = ' -> '.join(str(n) for n in data['route']) # [cite: 125]
        depot_id = data['route'][0] if data['route'] else 'N/A' # [cite: 125]

        # Calculate demands satisfied
        demands = [str(params['client_demand'][c]) for c in data['clients_served']] # [cite: 126]
        demands_str = '-'.join(demands) if demands else 'None' # [cite: 126]

        summary_data.append({
            'VehicleId': v_id, # [cite: 126]
            'DepotId': depot_id, # [cite: 126]
            'InitialLoad': data['load'], # [cite: 126]
            'RouteSequence': route_str, # [cite: 126]
            'ClientsServed': len(data['clients_served']), # [cite: 127]
            'DemandsSatisfied': demands_str, # [cite: 127]
            'TotalDistance': round(data['distance'], 2), # [cite: 127]
            'TotalTime': round(data['distance'] * 2, 1),  # Estimate: 2 min/km [cite: 127]
            'FuelCost': round(data['distance'] * params['total_unit_distance_cost'], 2) # [cite: 127]
        })

    if summary_data:
        summary_df = pd.DataFrame(summary_data) # [cite: 127]
        print(summary_df.to_string(index=False)) # [cite: 127]

        # --- NUEVA LÍNEA PARA GUARDAR EL CSV ---
        # Asegúrate de que el nombre del archivo corresponda al caso (ej. verificacion_caso2.csv)
        # El formato del archivo debe seguir estrictamente la estructura definida [cite: 60]
        # Los valores numéricos deben utilizar punto como separador decimal (comportamiento por defecto de to_csv) [cite: 71]
        # Las secuencias de valores deben separarse con guiones (-) sin espacios (ya manejado en demands_str y route_str) [cite: 72]
        # El archivo debe tener encabezados exactamente como se muestra en el ejemplo [cite: 73] (Pandas lo hace por defecto)
        nombre_archivo_verificacion = 'verificacion_caso2.csv'
        summary_df.to_csv(nombre_archivo_verificacion, index=False, sep=',') # Usamos ',' como separador estándar de CSV
        print(f"\nArchivo de verificación guardado como '{nombre_archivo_verificacion}'")
        # --- FIN DE LA NUEVA LÍNEA ---

    else:
        print("No active routes found.") # [cite: 128]

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("Starting VRP Case 2: Multiple Depots")

    depots_df = pd.read_csv('depots_case2.csv').head(3)  # Select first 3 depots
    clients_df = pd.read_csv('clients_case2.csv')
    vehicles_df = pd.read_csv('vehicles_case2.csv')

    print(f"Loaded: {str(len(depots_df))} depots, {str(len(clients_df))} clients, {str(len(vehicles_df))} vehicles")

    # Preprocess data
    print("\nPreprocessing data...")
    model_params = preprocess_data(depots_df, clients_df, vehicles_df, ORS_API_KEY)
    if model_params is None:
        print("Preprocessing failed.")
        exit()

    # Build and solve model
    print("\nBuilding model...")
    vrp_model = build_vrp_model(model_params)

    print("Solving model...")
    solver = SolverFactory('highs')  # Changed from 'glpk' to 'highs'
    results = solver.solve(vrp_model, tee=True)

    # Extract and display results
    solution = extract_routes_and_details(vrp_model, results, model_params)
    if solution:
        # Create visualization
        map_viz = visualize_routes_folium(solution, model_params)
        if map_viz:
            display(map_viz)
            map_viz.save('case2_routes.html')
            print("\nMap saved as 'case2_routes.html'")

        # Display summary
        display_vehicle_summary(solution, model_params)

        print(f"\nTotal solution cost: {solution['total_cost']:.2f} COP")
    else:
        print("No valid solution found.")

Starting VRP Case 2: Multiple Depots
Loaded: 3 depots, 9 clients, 6 vehicles

Preprocessing data...
Depots: ['D1.0', 'D2.0', 'D3.0']
Clients: ['C1.0', 'C2.0', 'C3.0', 'C4.0', 'C5.0', 'C6.0', 'C7.0', 'C8.0', 'C9.0']
All nodes: ['D1.0', 'D2.0', 'D3.0', 'C1.0', 'C2.0', 'C3.0', 'C4.0', 'C5.0', 'C6.0', 'C7.0', 'C8.0', 'C9.0']
Preprocessing complete: depots=3, clients=9, vehicles=6

Building model...
Solving model...
Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 732 rows; 918 cols; 4644 nonzeros; 864 integer variables (864 binary)
Coefficient ranges:
  Matrix [1e+00, 4e+01]
  Cost   [1e+04, 2e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
675 rows, 559 cols, 3317 nonzeros  0s
525 rows, 489 cols, 2726 nonzeros  0s
525 rows, 312 cols, 1679 nonzeros  0s
333 rows, 307 cols, 1689 nonzeros  0s
333 rows, 307 cols, 1689 nonzeros  0s

Solving MIP model with:
   333 rows
   307 cols (280 binary, 0 integer, 0 implied int., 27 


Map saved as 'case2_routes.html'

--- Vehicle Summary ---
 VehicleId DepotId  InitialLoad                                                                        RouteSequence  ClientsServed                           DemandsSatisfied  TotalDistance  TotalTime  FuelCost
         2    D1.0        101.0 D1.0 -> C5.0 -> C6.0 -> C4.0 -> C1.0 -> C2.0 -> C9.0 -> C7.0 -> C8.0 -> C3.0 -> D1.0              9 12.0-15.0-15.0-6.0-5.0-11.0-12.0-10.0-15.0          95.43      190.9 529455.18

Archivo de verificación guardado como 'verificacion_caso2.csv'

Total solution cost: 529455.18 COP


# CASO 3

In [ ]:
# -*- coding: utf-8 -*-
"""Etapa2.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1a-gbaRDw7S1R5z-aEFbj-m47mfjutnYz
"""

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    print("Installing IDAES, GLPK, IPOPT using helper script...")
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()
    print("Helper script finished.")

    # Añadir la instalación de HiGHS aquí
    print("Installing HiGHS...")
    !pip install highspy
    print("HiGHS installed.")
    print("instalando folium")
    !pip install folium
    print("folium instalado")

"""# Caso Base"""

import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

# --- 1. Data Generation (Simulating CSV files) ---
def generate_sample_data(num_depots, num_clients, num_vehicles):
    """Generates sample dataframes for depots, clients, and vehicles."""
    depots_data = {
        'DepotID': [f'D{i+1}' for i in range(num_depots)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_depots)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_depots)],
        'DepotCapacity': [random.randint(500, 1000) for _ in range(num_depots)]
    }
    depots_df = pd.DataFrame(depots_data)

    clients_data = {
        'ClientID': [f'C{i+1}' for i in range(num_clients)],
        'Demand': [random.randint(10, 50) for _ in range(num_clients)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_clients)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_clients)]
    }
    clients_df = pd.DataFrame(clients_data)

    vehicles_data = {
        'VehicleID': [f'V{i+1}' for i in range(num_vehicles)],
        'Capacity': [random.randint(80, 150) for _ in range(num_vehicles)],
        'Range': [random.randint(100, 300) for _ in range(num_vehicles)] # Range in km
    }
    vehicles_df = pd.DataFrame(vehicles_data)

    depots_df.to_csv('depots.csv', index=False)
    clients_df.to_csv('clients.csv', index=False)
    vehicles_df.to_csv('vehicles.csv', index=False)
    print("Generated sample data saved to CSVs (depots.csv, clients.csv, vehicles.csv).")

    return depots_df, clients_df, vehicles_df


# --- Función para obtener matriz de distancias/tiempos de la API ---
def get_matrix_from_ors_api(locations_dict, node_ids, api_key):
    """
    Obtiene la matriz de distancias y duraciones de la API de Openrouteservice.
    locations_dict: dict {node_id: (lon, lat)}
    node_ids: lista ordenada de node_ids (Depots + Clients)
    """
    api_locations = [[locations_dict[node_id][0], locations_dict[node_id][1]] for node_id in node_ids]

    body = {"locations": api_locations, "metrics": ["distance", "duration"]}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    print(f"Calling Openrouteservice API for matrix with {len(node_ids)} locations...")
    print(f"Requesting metrics: distance, duration.")

    try:
        response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)

        if response.status_code == 200:
            data = response.json()
            distance_matrix_meters = data.get("distances")
            duration_matrix_seconds = data.get("durations")

            if distance_matrix_meters and duration_matrix_seconds:
                print("API call successful. Matrix received.")
                distances_km = {}
                durations_sec = {}
                n = len(node_ids)
                for i in range(n):
                    for j in range(n):
                        from_node = node_ids[i]
                        to_node = node_ids[j]
                        dist_m = distance_matrix_meters[i][j] if distance_matrix_meters[i] and distance_matrix_meters[i][j] is not None else float('inf')
                        dur_s = duration_matrix_seconds[i][j] if duration_matrix_seconds[i] and duration_matrix_seconds[i][j] is not None else float('inf')

                        distances_km[(from_node, to_node)] = dist_m / 1000.0 # Convertir metros a km
                        durations_sec[(from_node, to_node)] = dur_s # Segundos

                return distances_km, durations_sec # Retornamos ambas matrices

            else:
                print("API response missing 'distances' or 'durations' data.")
                print(f"Response body: {response.text}")
                return None, None

        else:
            print(f"API call failed with status code {response.status_code}: {response.reason}")
            print(f"Response body: {response.text}")
            return None, None

    except requests.exceptions.Timeout:
        print(f"API request timed out after 60 seconds.")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        return None, None
    except json.JSONDecodeError:
        print(f"API response is not valid JSON. Response body: {response.text}")
        return None, None


# --- 2. Preprocessing and Parameter Calculation (Modificada) ---
def preprocess_data(depots_df, clients_df, vehicles_df, api_key):
    """Processes raw dataframes into dictionaries for Pyomo."""
    params = {}

    locations = {}
    for _, row in depots_df.iterrows():
        locations[row['DepotID']] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations[row['ClientID']] = (row['Longitude'], row['Latitude'])

    params['locations'] = locations

    all_nodes = depots_df['DepotID'].tolist() + clients_df['ClientID'].tolist()

    # Llama a la API para obtener las matrices de distancia y duración
    distances_km, durations_sec = get_matrix_from_ors_api(locations, all_nodes, api_key)

    if distances_km is None:
        print("Error getting distance/duration matrix from API. Cannot proceed.")
        return None

    params['distances'] = distances_km
    params['durations'] = durations_sec # Guardamos también las duraciones

    params['depot_capacity'] = dict(zip(depots_df['DepotID'], depots_df['DepotCapacity']))
    params['client_demand'] = dict(zip(clients_df['ClientID'], clients_df['Demand']))
    params['vehicle_capacity'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity']))
    params['vehicle_range'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Range']))

    params['F_freight_cost'] = 5000
    params['M_maintenance_cost'] = 23
    params['Pf_fuel_cost'] = 525.1
    params['total_unit_distance_cost'] = params['F_freight_cost'] + params['M_maintenance_cost'] + params['Pf_fuel_cost']

    params['set_D'] = depots_df['DepotID'].tolist()
    params['set_C'] = clients_df['ClientID'].tolist()
    params['set_V'] = vehicles_df['VehicleID'].tolist()
    params['set_N'] = params['set_D'] + params['set_C']

    return params

# --- 3. Pyomo Model Implementation ---
def build_vrp_model(params):
    """Builds the Pyomo VRP model."""
    model = pyo.ConcreteModel(name="VehicleRoutingProblem")

    # --- SETS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])
    model.C_plus_D = model.C | model.D

    # --- PARAMETERS ---
    model.dist = pyo.Param(model.N, model.N, initialize=params['distances'])
    # model.duration = pyo.Param(model.N, model.N, initialize=params['durations']) # Descomentar si necesitas duraciones en las restricciones

    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.cost_per_km = pyo.Param(initialize=params['total_unit_distance_cost'])
    model.num_clients = pyo.Param(initialize=len(params['set_C']))

    # --- DECISION VARIABLES ---
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary, rule=lambda m, c1, c2, k: m.x_c1c2k[c1,c2,k].setlb(0) if c1==c2 else None )
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)
    model.u_jk = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals)

    # --- OBJECTIVE FUNCTION (Minimize total distance cost) ---
    def objective_rule(m):
        cost = 0
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D for k in m.V)
        return cost
    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- CONSTRAINTS ---
    # 1. Client Assignment: Each client j is assigned to exactly one vehicle k.
    def client_assignment_rule(m, j):
        return sum(m.Y_jk[j,k] for k in m.V) == 1
    model.client_assignment_con = pyo.Constraint(model.C, rule=client_assignment_rule)

    # 2. Vehicle Capacity: Total demand of clients assigned to vehicle k must not exceed its capacity.
    def vehicle_capacity_rule(m, k):
        return sum(m.client_demand[j] * m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    model.vehicle_capacity_con = pyo.Constraint(model.V, rule=vehicle_capacity_rule)

    # 3a: A vehicle starts from at most one depot to at most one client.
    def vehicle_must_start_if_used_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) <= 1
    model.vehicle_start_at_most_one_depot_con = pyo.Constraint(model.V, rule=vehicle_must_start_if_used_rule)

    # 3b: If a vehicle is used (serves any client), it must start from a depot.
    def link_Y_to_z_start_rule(m, k):
         M = m.num_clients
         return sum(m.Y_jk[j,k] for j in m.C) <= M * sum(m.z_djk[d,j,k] for d in m.D for j in m.C)
    model.link_Y_to_z_start_con = pyo.Constraint(model.V, rule=link_Y_to_z_start_rule)

    # 4. Route End: If a vehicle starts a tour, it must end at a depot.
    def vehicle_tour_end_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) == sum(m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
    model.vehicle_tour_end_con = pyo.Constraint(model.V, rule=vehicle_tour_end_rule)

    # 5. Flow Conservation at Client Nodes for vehicle k:
    def flow_conservation_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        outgoing_from_j = sum(m.x_prime_cdk[j_node,d,k] for d in m.D) + sum(m.x_c1c2k[j_node,c2,k] for c2 in m.C if j_node != c2)
        return incoming_to_j == outgoing_from_j
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_conservation_rule)

    # 6. Linking Y_jk (Assignment) to actual visit for vehicle k:
    def link_assign_to_visit_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        return incoming_to_j == m.Y_jk[j_node,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_to_visit_rule)

    # 7. Depot Capacity: Total demand leaving depot d as first stop.
    def depot_capacity_rule_pdf(m, d_node):
        return sum(m.client_demand[j] * m.z_djk[d_node,j,k] for j in m.C for k in m.V) <= m.depot_cap[d_node]
    model.depot_capacity_pdf_con = pyo.Constraint(model.D, rule=depot_capacity_rule_pdf)

    # 8. Vehicle Range (Uses real distances from model.dist)
    def vehicle_range_rule(m,k):
        dist_k = 0
        dist_k += sum(m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C)
        dist_k += sum(m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2)
        dist_k += sum(m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return dist_k <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 9. No self-loops for x_c1c2k (client to same client)
    def no_self_loops_clients_rule(m, c, k):
        return m.x_c1c2k[c,c,k] == 0
    model.no_self_loops_clients_con = pyo.Constraint(model.C, model.V, rule=no_self_loops_clients_rule)

    # 10. Subtour Elimination (MTZ Constraints)
    def u_bounds_lower_rule(m, j, k):
        return m.u_jk[j,k] >= 1.0 * m.Y_jk[j,k]
    model.u_bounds_lower_con = pyo.Constraint(model.C, model.V, rule=u_bounds_lower_rule)

    def u_bounds_upper_rule(m, j, k):
        return m.u_jk[j,k] <= (m.num_clients) * m.Y_jk[j,k]
    model.u_bounds_upper_con = pyo.Constraint(model.C, model.V, rule=u_bounds_upper_rule)

    def mtz_rule(m, c1, c2, k):
        if c1 == c2:
            return pyo.Constraint.Skip
        return m.u_jk[c1,k] - m.u_jk[c2,k] + (m.num_clients) * m.x_c1c2k[c1,c2,k] <= (m.num_clients) - 1.0
    model.mtz_con = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

    def mtz_initial_u_rule(m, d, j, k):
        return m.u_jk[j,k] <= 1.0 + (m.num_clients) * (1 - m.z_djk[d,j,k])
    model.mtz_initial_u_con = pyo.Constraint(model.D, model.C, model.V, rule=mtz_initial_u_rule)

    return model

# --- 4. Solving the Model and Extracting Results ---
def solve_model(model, solver_name='cbc', time_limit_seconds=None):
    """Solves the Pyomo model and returns results."""
    solver = SolverFactory(solver_name)

    if time_limit_seconds is not None:
        print(f"Setting time limit for solver '{solver_name}' to {time_limit_seconds} seconds.")
        if solver_name.lower() == 'gurobi':
            solver.options['TimeLimit'] = time_limit_seconds
        elif solver_name.lower() == 'cbc':
             solver.options['sec'] = time_limit_seconds
        elif solver_name.lower() == 'highs':
            solver.options['time_limit'] = time_limit_seconds
        else:
            print(f"Warning: Time limit option not known for solver '{solver_name}'. Time limit not set.")

    print(f"Attempting to solve with {solver_name}...")
    results = solver.solve(model, tee=True)

    return results

def extract_routes_and_details(model, results, params):
    """Extracts routes, loads, and costs from the solved model."""
    if not (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) or \
       not (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):
        print(f"Solver did not find an acceptable solution. Status: {results.solver.status}, Condition: {results.solver.termination_condition}")
        return None

    print("\n--- Solution Found ---")
    try:
        objective_value = pyo.value(model.total_cost)
        print(f"Total Cost: {objective_value:.2f} COP")
        solution_details = {'total_cost': objective_value, 'routes': {}}
    except Exception as e:
        print(f"Warning: Could not access objective value from model after solve: {e}.")
        print("Attempting to extract routes based on variable values...")
        solution_details = {'total_cost': float('inf'), 'routes': {}}


    for k_id in params['set_V']:
        vehicle_route = []
        vehicle_load = 0 # Calculated at the end
        vehicle_dist = 0
        current_node = None
        is_vehicle_used_for_route = False

        # Check start from depot (z_djk)
        for d_id in params['set_D']:
            for j_id in params['set_C']:
                 if pyo.value(model.z_djk[d_id,j_id,k_id]) is not None and pyo.value(model.z_djk[d_id,j_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    vehicle_route.append(j_id)
                    current_node = j_id
                    if (d_id, j_id) in params['distances']:
                         vehicle_dist += params['distances'][(d_id,j_id)]
                    else:
                         print(f"Warning: Distance not found for ({d_id}, {j_id}).")

                    is_vehicle_used_for_route = True
                    break
            if current_node: break

        if not is_vehicle_used_for_route:
            solution_details['routes'][k_id] = {'route': [], 'load': 0, 'distance': 0, 'clients_served': []}
            continue

        # Follow path between clients (x_c1c2k)
        clients_assigned_to_k = [j_id for j_id in params['set_C']
                                 if pyo.value(model.Y_jk[j_id,k_id]) is not None
                                 and pyo.value(model.Y_jk[j_id,k_id]) > 0.5]


        safety_counter = 0
        max_safety_steps = len(params['set_C']) * 2

        while current_node and current_node in params['set_C'] and safety_counter < max_safety_steps :

            found_next_client = False
            next_node = None

            for c2_id in params['set_C']:
                if c2_id != current_node:
                     if pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) is not None and pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) > 0.5:

                         if c2_id in vehicle_route:
                             print(f"Warning: Client {c2_id} would be revisited by {k_id} in current trace. Route: {vehicle_route}. Stopping trace for this vehicle.")
                             current_node = None
                             found_next_client = False
                             break

                         next_node = c2_id
                         found_next_client = True
                         break

            if found_next_client and next_node:
                 vehicle_route.append(next_node)
                 if (current_node, next_node) in params['distances']:
                     vehicle_dist += params['distances'][(current_node, next_node)]
                 else:
                     print(f"Warning: Distance not found for ({current_node}, {next_node}).")

                 current_node = next_node
                 safety_counter += 1

            else:
                break

        # Check for return to depot (x_prime_cdk) from the last client visited
        returned_to_depot = False
        last_node_in_trace = vehicle_route[-1] if vehicle_route else None

        if last_node_in_trace and last_node_in_trace in params['set_C']:
             for d_id in params['set_D']:
                 if pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) is not None and pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    if (last_node_in_trace, d_id) in params['distances']:
                         vehicle_dist += params['distances'][(last_node_in_trace,d_id)]
                    else:
                         print(f"Warning: Distance not found for ({last_node_in_trace}, {d_id}).")

                    returned_to_depot = True
                    break

        # Final load calculation based on Y_jk (clients assigned to this vehicle)
        actual_load_for_vehicle = sum(params['client_demand'][c] for c in clients_assigned_to_k)

        # Store the extracted route details
        solution_details['routes'][k_id] = {
            'route': vehicle_route,
            'load': actual_load_for_vehicle,
            'distance': vehicle_dist,
            'clients_served': clients_assigned_to_k
        }

        # Add warnings if the extracted route seems incomplete or problematic
        if is_vehicle_used_for_route:
             clients_in_traced_route = [node for node in vehicle_route if node in params['set_C']]
             assigned_but_not_traced = [c for c in clients_assigned_to_k if c not in clients_in_traced_route]
             traced_but_not_assigned = [c for c in clients_in_traced_route if c not in clients_assigned_to_k]
             if assigned_but_not_traced:
                  print(f"Warning: Vehicle {k_id} was assigned clients {assigned_but_not_traced} but did not visit them in trace.")
             if traced_but_not_assigned:
                  print(f"Warning: Vehicle {k_id} visited clients {traced_but_not_assigned} in trace which are not assigned to it.")

             route_starts_at_depot = len(vehicle_route) > 0 and vehicle_route[0] in params['set_D']
             route_ends_at_depot = len(vehicle_route) > 0 and vehicle_route[-1] in params['set_D']

             if is_vehicle_used_for_route and not (route_starts_at_depot and route_ends_at_depot):
                 if len(vehicle_route) > 1:
                      print(f"Warning: Vehicle {k_id} was used but its traced route does not start and end at a depot. Route: {' -> '.join(vehicle_route)}")


    return solution_details

# --- 5. Visualization (NetworkX) ---
# CORREGIDA: Usar la clave correcta 'depot_capacity'
def visualize_routes(solution_details, params):
    """Visualizes the routes on a graph."""
    if not solution_details or not solution_details['routes']:
        print("No solution to visualize.")
        return

    loc_data = params['locations']
    G = nx.DiGraph()

    pos = {}
    node_colors = {}
    node_labels = {}
    node_size = {}

    for d_id in params['set_D']:
        G.add_node(d_id)
        pos[d_id] = (loc_data[d_id][0], loc_data[d_id][1])
        node_colors[d_id] = 'red'
        node_labels[d_id] = f"{d_id}\n(Cap:{params['depot_capacity'][d_id]})"
        node_size[d_id] = 3000

    for c_id in params['set_C']:
        G.add_node(c_id)
        pos[c_id] = (loc_data[c_id][0], loc_data[c_id][1])
        node_colors[c_id] = 'lightblue'
        node_labels[c_id] = f"{c_id}\n(D:{params['client_demand'][c_id]})"
        node_size[c_id] = 1500 + params['client_demand'][c_id] * 20


    plt.figure(figsize=(16, 12))
    nx.draw_networkx_nodes(G, pos,
                             node_color=[node_colors[n] for n in G.nodes()],
                             node_size=[node_size[n] for n in G.nodes()],
                             alpha=0.9, edgecolors='black', linewidths=1)

    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=7, font_weight='bold')


    edge_colors = ['b', 'g', 'c', 'm', 'y', 'k', '#ff7f0e', '#d62728']
    vehicle_styles = ['-', '--', '-.', ':']

    for node_id in params['set_D'] + params['set_C']:
        if node_id not in G.nodes():
            print(f"Warning: Node {node_id} missing from graph before drawing edges.")
            if node_id in loc_data:
                 pos[node_id] = (loc_data[node_id][0], loc_data[node_id][1])
                 G.add_node(node_id)

    print("\n--- Visualized Routes (NetworkX) ---")
    vehicle_idx = 0
    vehicles_with_routes = []

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            vehicles_with_routes.append(v_id)
            color = edge_colors[vehicle_idx % len(edge_colors)]
            style = vehicle_styles[vehicle_idx % len(vehicle_styles)]

            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]

                if u_node in G.nodes() and v_node in G.nodes():
                    G.add_edge(u_node, v_node, color=color, style=style, vehicle=v_id)
                else:
                    print(f"Warning: Skipping edge ({u_node} -> {v_node}) for Vehicle {v_id} as one or both nodes are not in the graph.")

            vehicle_idx += 1

    edge_list_with_attributes = []
    for u, v, data in G.edges(data=True):
        edge_list_with_attributes.append(((u, v), data.get('color', 'gray'), data.get('style', '-')))

    if edge_list_with_attributes:
         edges_to_draw = [edge[0] for edge in edge_list_with_attributes]
         colors_for_draw = [edge[1] for edge in edge_list_with_attributes]
         styles_for_draw = [edge[2] for edge in edge_list_with_attributes]

         pos_for_edges = {node: pos[node] for node in G.nodes() if node in pos}

         if pos_for_edges:
             nx.draw_networkx_edges(G, pos_for_edges,
                                    edgelist=edges_to_draw,
                                    edge_color=colors_for_draw,
                                    style=styles_for_draw,
                                    width=2.0,
                                    alpha=0.7,
                                    arrowstyle='->',
                                    arrows=True,
                                    node_size=[node_size.get(node, 1000) for node in pos_for_edges.keys()],
                                    arrowsize=20
                                    )
         else:
             print("Warning: No node positions available to draw edges.")

    handles = []
    labels = []
    vehicle_idx = 0

    for v_id in vehicles_with_routes:
         data = solution_details['routes'][v_id]
         color = edge_colors[vehicle_idx % len(edge_colors)]
         style = vehicle_styles[vehicle_idx % len(vehicle_styles)]
         handles.append(plt.Line2D([0], [0], color=color, lw=2, linestyle=style))
         labels.append(f"Veh {v_id}: Load {data['load']:.0f}/{params['vehicle_capacity'][v_id]}, Dist {data['distance']:.1f}/{params['vehicle_range'][v_id]:.1f} km")
         vehicle_idx += 1


    if handles:
        plt.legend(handles, labels, title="Vehicle Routes", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
        plt.subplots_adjust(right=0.7)


    plt.title("Vehicle Routes (from Openrouteservice API distances)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.show()


# --- New Function for Folium Visualization ---
def visualize_routes_folium(solution_details, params):
    """Visualizes the routes on an interactive Folium map."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution to visualize with Folium.")
        return None

    print("\nGenerating Folium map...")

    loc_data = params['locations'] # {node_id: (lon, lat)}

    # Determine map center - use the first depot's location as a simple center
    # Folium expects [latitude, longitude]
    map_center = [loc_data[params['set_D'][0]][1], loc_data[params['set_D'][0]][0]] if params['set_D'] else [4.6, -74.0] # Fallback for Bogotá

    # Create base map
    m = folium.Map(location=map_center, zoom_start=12) # Adjust zoom as needed

    # Add Depot Markers
    for d_id in params['set_D']:
        lat, lon = loc_data[d_id][1], loc_data[d_id][0] # Folium expects [lat, lon]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Depot {d_id}</b><br>Capacity: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home') # Use a home icon for depots
        ).add_to(m)

    # Add Client Markers
    for c_id in params['set_C']:
        lat, lon = loc_data[c_id][1], loc_data[c_id][0]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Client {c_id}</b><br>Demand: {params['client_demand'][c_id]}",
            icon=folium.Icon(color='blue', icon='info-sign') # Use info-sign for clients
        ).add_to(m)

    # Add Vehicle Routes as Polylines
    route_colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue', 'darkgreen', 'darkpurple']

    vehicle_idx = 0
    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            route_coords = []
            for node_id in route:
                 if node_id in loc_data:
                      route_coords.append([loc_data[node_id][1], loc_data[node_id][0]])
                 else:
                      print(f"Warning: Location data missing for node {node_id} in route for Vehicle {v_id}. Skipping polyline segment.")
                      pass


            if len(route_coords) > 1:
                color = route_colors[vehicle_idx % len(route_colors)]
                popup_text = f"<b>Vehicle {v_id}</b><br>"
                popup_text += f"Route: {' -> '.join(route)}<br>"
                popup_text += f"Load: {data['load']:.0f}/{params['vehicle_capacity'][v_id]}<br>"
                popup_text += f"Distance: {data['distance']:.2f} km"

                folium.PolyLine(
                    route_coords,
                    color=color,
                    weight=2.5,
                    opacity=0.8,
                    popup=popup_text
                ).add_to(m)

                vehicle_idx += 1

    print("Folium map generated.")
    return m

# --- New Function to Display Vehicle Summary ---
def display_vehicle_summary(solution_details, model_params):
    """Displays a summary table for each vehicle."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution details available to display vehicle summary.")
        return

    print("\n--- Vehicle Summary ---")

    summary_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available in model_params. Cannot calculate total time.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)
    if 'Pf_fuel_cost' not in model_params:
         print("Warning: 'Pf_fuel_cost' not found in model_params. Fuel cost will be 0.")


    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        demands_satisfied = initial_load

        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        summary_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': f"{initial_load:.0f}",
            'RouteSequence': route_sequence_str,
            'ClientsServed': ", ".join(clients_served) if clients_served else "None",
            'DemandsSatisfied': f"{demands_satisfied:.0f}",
            'TotalDistance (km)': f"{total_distance:.2f}",
            'TotalTime (s)': f"{total_time_sec:.0f}",
            'FuelCost (COP)': f"{calculated_fuel_cost:.2f}"
        })

    summary_df = pd.DataFrame(summary_list)

    if not summary_df.empty:
        print(summary_df.to_string(index=False))
    else:
        print("No vehicle summary data generated.")


# --- New Function to Save Verification File ---
def save_verification_file(solution_details, model_params, case_number):
    """Saves the vehicle summary to a verification CSV file."""
    if not solution_details or not solution_details['routes']:
        print(f"\nNo solution details available to save verification file for case {case_number}.")
        return

    print(f"\nGenerating verification file for case {case_number}...")

    verification_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available. Total time in verification file will be 0 or inaccurate.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served_list = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        clients_served_count = len(clients_served_list)

        # Calculate Demands Satisfied (list of demands in order of visit)
        demands_satisfied_list = []
        if route:
            for node_id in route:
                # Only include demands for client nodes that are actually in the route
                if node_id in model_params['set_C']:
                    demands_satisfied_list.append(str(model_params['client_demand'][node_id]))
        demands_satisfied_str = "-".join(demands_satisfied_list) if demands_satisfied_list else "None"

        # Calculate Total Time from the route sequence and durations matrix
        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    # print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        # Convert total time from seconds to minutes
        total_time_min = total_time_sec / 60.0

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        verification_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': int(initial_load), # Should be integer demand sum
            'RouteSequence': route_sequence_str,
            'ClientsServed': clients_served_count,
            'DemandsSatisfied': demands_satisfied_str,
            'TotalDistance': round(total_distance, 2), # Round to 2 decimal places
            'TotalTime': round(total_time_min, 1), # Round to 1 decimal place for minutes
            'FuelCost': round(calculated_fuel_cost, 2) # Round to 2 decimal places
        })

    verification_df = pd.DataFrame(verification_list)

    if not verification_df.empty:
        # Ensure columns are in the specified order
        csv_columns_order = [
            'VehicleId',
            'DepotId',
            'InitialLoad',
            'RouteSequence',
            'ClientsServed',
            'DemandsSatisfied',
            'TotalDistance',
            'TotalTime',
            'FuelCost'
        ]
        verification_df = verification_df[csv_columns_order]

        # Define the filename
        filename = f"verificacion_caso{case_number}.csv"

        # Save the DataFrame to CSV
        try:
            verification_df.to_csv(filename, index=False)
            print(f"Verification file saved successfully: {filename}")
        except Exception as e:
            print(f"Error saving verification file {filename}: {e}")

    else:
        print(f"No verification data generated for case {case_number}.")


# --- Main Execution ---
if __name__ == "__main__":
    print("Starting VRP Model with Openrouteservice API distances.")
    num_depots = 1
    num_clients = 24
    num_vehicles = 8
    print(f"Problem size: {num_depots} depots, {num_clients} clients, {num_vehicles} vehicles.")

    # Define the case number for the verification file
    current_case_number = 1 # You can change this for different cases

    depots_df, clients_df, vehicles_df = None, None, None
    try:
        # Check if CSVs exist, if not, sample data will be generated and saved
        depots_df = pd.read_csv('depots.csv')
        clients_df = pd.read_csv('clients.csv')
        vehicles_df = pd.read_csv('vehicles.csv')
        print("Data loaded from CSVs.")
    except FileNotFoundError:
        print("CSV files not found (depots.csv, clients.csv, vehicles.csv). Generating sample data instead.")
        depots_df, clients_df, vehicles_df = generate_sample_data(num_depots, num_clients, num_vehicles)

    if depots_df is None or clients_df is None or vehicles_df is None:
        print("Failed to load or generate data. Exiting.")
        exit()

    print("\nPreprocessing data (calling API for distances)...")
    model_params = preprocess_data(depots_df, clients_df, vehicles_df, ORS_API_KEY)

    if model_params is None:
        print("Preprocessing failed (API error?). Exiting.")
        exit()

    print("\nBuilding Pyomo model...")
    vrp_model = build_vrp_model(model_params)
    print("Model built.")

    print("\nSolving model (this might take time depending on size and solver)...")
    solver_name = 'highs'
    solve_time_limit = 900

    try:
        results = solve_model(vrp_model, solver_name=solver_name, time_limit_seconds=solve_time_limit)

        if (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) and \
           (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):

            print(f"\nSolver terminated with status: {results.solver.status} and condition: {results.solver.termination_condition}")

            if results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
                 print("Result: Solver reached time limit. Reporting the best feasible solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.optimal:
                 print("Result: Optimal solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
                 print("Result: Feasible solution found (optimality not proven).")

            try:
                 objective_value = pyo.value(vrp_model.total_cost)
                 print(f"\nBest feasible objective value found: {objective_value:.2f} COP")
            except Exception as e:
                 print(f"\nWarning: Could not access objective value from model after successful solve status: {e}")
                 print("Proceeding to extract routes based on variable values...")

            extracted_solution = extract_routes_and_details(vrp_model, results, model_params)

            if extracted_solution:
                # Visualize using NetworkX (your existing method) - Comentada
                #print("\nVisualizing routes (NetworkX)...")
                #visualize_routes(extracted_solution, model_params) # Ensure this function is corrected in your file

                # Visualize using Folium
                print("\nVisualizing routes (Folium)...")
                folium_map = visualize_routes_folium(extracted_solution, model_params)
                if folium_map:
                    display(folium_map)

                # Display vehicle summary table
                display_vehicle_summary(extracted_solution, model_params)

                # NEW: Save verification file
                save_verification_file(extracted_solution, model_params, case_number=current_case_number)


            else:
                print("No solution details extracted.")

        elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
            print("\nSolver terminated with condition: Infeasible.")
            print("Model is infeasible. Check constraints and data.")

        else:
            print(f"\nSolver terminated with unexpected status: {results.solver.status}, condition: {results.solver.termination_condition}")
            print("Solver did not find an acceptable solution.")
            print("Check solver output above for details.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during model setup, solving, or post-processing: {e}")
        print(f"Ensure solver '{solver_name}' is installed and accessible.")
        print("Verify data loading, preprocessing, and model building steps if error occurred early.")

"""# CASO 2"""

import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

def get_matrix_from_ors_api(locations, nodes, api_key):
    """Calls OpenRouteService Matrix API and returns distance and duration matrices."""
    coords = [locations[node] for node in nodes]
    body = {"locations": coords, "metrics": ["distance", "duration"], "units": "km"}
    headers = {'Authorization': api_key}
    response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)
    if response.status_code == 200:
        data = response.json()
        raw_dist = data['distances']
        raw_dur = data['durations']
    else:
        print(f"ORS API call failed with status code {response.status_code}: {response.text}")
        return None, None
    return raw_dist, raw_dur

def preprocess_data(depots_df, clients_df, vehicles_df, api_key):
    """
    Processes raw dataframes into dictionaries for Pyomo.
    Returns None on API failure.
    """
    # --- 1. Create unique node identifiers to avoid conflicts ---
    # Prefix depot IDs with 'D' and client IDs with 'C'
    depots = ['D' + str(row['DepotID']) for _, row in depots_df.iterrows()]
    clients = ['C' + str(row['ClientID']) for _, row in clients_df.iterrows()]
    vehicles = vehicles_df['VehicleID'].tolist()
    all_nodes = depots + clients

    print("Depots:", depots)
    print("Clients:", clients)
    print("All nodes:", all_nodes)

    # Create locations dictionary with new identifiers
    locations = {}
    for _, row in depots_df.iterrows():
        locations['D' + str(row['DepotID'])] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations['C' + str(row['ClientID'])] = (row['Longitude'], row['Latitude'])

    # --- 2. Call ORS API ---
    raw_dist, raw_dur = get_matrix_from_ors_api(locations, all_nodes, api_key)
    if raw_dist is None:
        print("Error getting distance/duration matrix from API.")
        return None

    # Build distance matrices with new identifiers
    dist_dc = {}; dur_dc = {}
    dist_cc = {}; dur_cc = {}
    dist_cd = {}; dur_cd = {}

    for i, d in enumerate(depots):
        for j, c in enumerate(clients):
            dist_dc[(d, c)] = raw_dist[i][j + len(depots)]
            dur_dc[(d, c)] = raw_dur[i][j + len(depots)]

    for i, c1 in enumerate(clients):
        for j, c2 in enumerate(clients):
            dist_cc[(c1, c2)] = raw_dist[i + len(depots)][j + len(depots)]
            dur_cc[(c1, c2)] = raw_dur[i + len(depots)][j + len(depots)]

    for i, c in enumerate(clients):
        for j, d in enumerate(depots):
            dist_cd[(c, d)] = raw_dist[i + len(depots)][j]
            dur_cd[(c, d)] = raw_dur[i + len(depots)][j]

    # Create unified distance dictionary
    all_distances = {}
    all_distances.update(dist_dc)
    all_distances.update(dist_cc)
    all_distances.update(dist_cd)

    # Create parameter dictionaries with new identifiers
    depot_capacity = {}
    for _, row in depots_df.iterrows():
        depot_capacity['D' + str(row['DepotID'])] = row['Capacity']

    client_demand = {}
    for _, row in clients_df.iterrows():
        client_demand['C' + str(row['ClientID'])] = row['Demand']

    params = {
        'locations': locations,
        'dist_dc': dist_dc, 'dist_cc': dist_cc, 'dist_cd': dist_cd,
        'dur_dc': dur_dc, 'dur_cc': dur_cc, 'dur_cd': dur_cd,
        'distances': all_distances,
        'depot_capacity': depot_capacity,
        'client_demand': client_demand,
        'vehicle_capacity': dict(zip(vehicles, vehicles_df['Capacity'])),
        'vehicle_range': dict(zip(vehicles, vehicles_df['Range'])),
        'F_freight_cost': 5000,
        'M_maintenance_cost': 23,
        'Pf_fuel_cost': 525.1,
    }
    params['total_unit_distance_cost'] = (
        params['F_freight_cost']
      + params['M_maintenance_cost']
      + params['Pf_fuel_cost']
    )
    params['set_D'] = depots
    params['set_C'] = clients
    params['set_V'] = vehicles
    params['set_N'] = all_nodes

    print(f"Preprocessing complete: depots={len(depots)}, clients={len(clients)}, vehicles={len(vehicles)}")
    return params

def build_vrp_model(params):
    """Builds the Pyomo VRP model."""
    model = pyo.ConcreteModel(name="VehicleRoutingProblem")

    # --- SETS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])

    # --- PARAMETERS ---
    # Create full distance matrix for all node pairs
    full_dist = {}
    for n1 in params['set_N']:
        for n2 in params['set_N']:
            if (n1, n2) in params['distances']:
                full_dist[(n1, n2)] = params['distances'][(n1, n2)]
            elif n1 == n2:
                full_dist[(n1, n2)] = 0
            else:
                # This shouldn't happen if preprocessing is correct
                full_dist[(n1, n2)] = 9999

    model.dist = pyo.Param(model.N, model.N, initialize=full_dist)
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.num_clients = len(params['set_C'])
    model.cost_per_km = params['total_unit_distance_cost']

    # --- VARIABLES ---
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary)
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)
    model.u_jk = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals)

    # --- OBJECTIVE FUNCTION ---
    def objective_rule(m):
        cost = 0
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k]
                    for d in m.D for j in m.C for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k]
                    for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k]
                    for c in m.C for d in m.D for k in m.V)
        return cost
    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- CONSTRAINTS ---
    # 1. Each client visited exactly once
    model.client_assignment_con = pyo.Constraint(
        model.C,
        rule=lambda m, j: sum(m.Y_jk[j,k] for k in m.V) == 1
    )

    # 2. Vehicle capacity
    model.vehicle_capacity_con = pyo.Constraint(
        model.V,
        rule=lambda m, k: sum(m.client_demand[j]*m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    )

    # 3. Flow conservation
    def flow_cons_rule(m, j, k):
        in_to_j = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j)
        out_from_j = sum(m.x_prime_cdk[j,d,k] for d in m.D) + sum(m.x_c1c2k[j,c2,k] for c2 in m.C if c2 != j)
        return in_to_j == out_from_j
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_cons_rule)

    # 4. Link assignment to visit
    def link_assign_visit_rule(m, j, k):
        in_to_j = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j)
        return in_to_j == m.Y_jk[j,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_visit_rule)

    # 5. Depot capacity
    model.depot_capacity_con = pyo.Constraint(
        model.D,
        rule=lambda m, d: sum(m.client_demand[j]*m.z_djk[d,j,k] for j in m.C for k in m.V) <= m.depot_cap[d]
    )

    # 6. Vehicle range
    def vehicle_range_rule(m, k):
        tot = sum(m.dist[d,j]*m.z_djk[d,j,k] for d in m.D for j in m.C)
        tot += sum(m.dist[c1,c2]*m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2)
        tot += sum(m.dist[c,d]*m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return tot <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 7. No self loops
    model.no_self_loops_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, c, k: m.x_c1c2k[c,c,k] == 0
    )

    # 8. MTZ subtour elimination
    model.u_bounds_lower_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, j, k: m.u_jk[j,k] >= m.Y_jk[j,k]
    )
    model.u_bounds_upper_con = pyo.Constraint(
        model.C, model.V, rule=lambda m, j, k: m.u_jk[j,k] <= m.num_clients * m.Y_jk[j,k]
    )

    # 9. Si un vehículo es usado, debe regresar a algún depósito
    def return_to_depot_if_used_rule(m, k):
        clients_served = sum(m.Y_jk[j, k] for j in m.C)
        depot_returns = sum(m.x_prime_cdk[j, d, k] for j in m.C for d in m.D)
        return clients_served <= m.num_clients * depot_returns
    model.return_to_depot_con = pyo.Constraint(model.V, rule=return_to_depot_if_used_rule)

    # 10. MTZ subtour elimination constraint
    def mtz_rule(m, c1, c2, k):
        if c1 == c2:
            return pyo.Constraint.Skip
        return (m.u_jk[c1,k] - m.u_jk[c2,k] + m.num_clients * m.x_c1c2k[c1,c2,k] <= m.num_clients - 1)
    model.mtz_con = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

    return model

def extract_routes_and_details(model, results, params):
    """Extracts routes from solved model with complete debugging."""
    if results.solver.termination_condition not in {
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.feasible,
            pyo.TerminationCondition.maxTimeLimit}:
        print(f"Solver did not find solution. Status: {results.solver.status}")
        return None

    print("\n--- Solution Found ---")
    try:
        objective_value = pyo.value(model.total_cost)
        print(f"Total Cost: {objective_value:.2f} COP")
    except:
        objective_value = float('inf')

    solution_details = {'total_cost': objective_value, 'routes': {}}

    for k_id in params['set_V']:
        print(f"\n=== ANALYZING VEHICLE {k_id} ===")

        # Find which clients are served by this vehicle
        clients_served = []
        for c_id in params['set_C']:
            if pyo.value(model.Y_jk[c_id, k_id]) and pyo.value(model.Y_jk[c_id, k_id]) > 0.5:
                clients_served.append(c_id)

        print(f"Clients served by vehicle {k_id}: {clients_served}")

        if not clients_served:
            solution_details['routes'][k_id] = {
                'route': [], 'load': 0, 'distance': 0, 'clients_served': []
            }
            continue

        # Debug all active connections for this vehicle
        print(f"\n--- All active connections for vehicle {k_id} ---")

        # Depot to client connections
        depot_client_connections = []
        for d_id in params['set_D']:
            for c_id in params['set_C']:
                val = pyo.value(model.z_djk[d_id, c_id, k_id])
                if val and val > 0.5:
                    depot_client_connections.append((d_id, c_id))
                    print(f"Depot->Client: {d_id} -> {c_id}")

        # Client to client connections
        client_client_connections = []
        for c1_id in params['set_C']:
            for c2_id in params['set_C']:
                if c1_id != c2_id:
                    val = pyo.value(model.x_c1c2k[c1_id, c2_id, k_id])
                    if val and val > 0.5:
                        client_client_connections.append((c1_id, c2_id))
                        print(f"Client->Client: {c1_id} -> {c2_id}")

        # Client to depot connections
        client_depot_connections = []
        for c_id in params['set_C']:
            for d_id in params['set_D']:
                val = pyo.value(model.x_prime_cdk[c_id, d_id, k_id])
                if val and val > 0.5:
                    client_depot_connections.append((c_id, d_id))
                    print(f"Client->Depot: {c_id} -> {d_id}")

        # Build route using graph traversal approach
        if not depot_client_connections:
            print(f"No depot-client connections found for vehicle {k_id}")
            continue

        # Find the starting depot and first client
        start_depot, first_client = depot_client_connections[0]
        route = [start_depot, first_client]
        current_node = first_client

        print(f"Starting route: {start_depot} -> {first_client}")

        # Follow the path through client-client connections
        visited_clients = {first_client}
        while len(visited_clients) < len(clients_served):
            next_node = None
            for c1, c2 in client_client_connections:
                if c1 == current_node and c2 not in visited_clients:
                    next_node = c2
                    break

            if next_node:
                route.append(next_node)
                visited_clients.add(next_node)
                current_node = next_node
                print(f"Added to route: -> {next_node}")
            else:
                print(f"No more client connections from {current_node}")
                break

        # Find the return to depot from the last client
        end_depot = None
        for c_id, d_id in client_depot_connections:
            if c_id == current_node:
                end_depot = d_id
                route.append(end_depot)
                print(f"Return to depot: {current_node} -> {end_depot}")
                break

        if not end_depot:
            print(f"WARNING: No return to depot found from {current_node}")

        # Calculate metrics
        total_load = sum(params['client_demand'][c] for c in clients_served)
        total_distance = 0
        for i in range(len(route) - 1):
            if (route[i], route[i+1]) in params['distances']:
                distance_segment = params['distances'][(route[i], route[i+1])]
                total_distance += distance_segment
                print(f"Distance {route[i]} -> {route[i+1]}: {distance_segment:.2f}km")
            else:
                print(f"Warning: Distance not found for {route[i]} -> {route[i+1]}")

        solution_details['routes'][k_id] = {
            'route': route,
            'load': total_load,
            'distance': total_distance,
            'clients_served': clients_served
        }

        print(f"\nFINAL ROUTE for Vehicle {k_id}: {' -> '.join(str(n) for n in route)}")
        print(f"Load: {total_load}, Distance: {total_distance:.1f}km")

    return solution_details

def visualize_routes_folium(solution_details, params):
    """Creates interactive Folium map."""
    if not solution_details or not solution_details['routes']:
        print("No solution to visualize.")
        return None

    loc_data = params['locations']

    # Center map
    all_lats = [loc[1] for loc in loc_data.values()]
    all_lons = [loc[0] for loc in loc_data.values()]
    center_lat = sum(all_lats) / len(all_lats)
    center_lon = sum(all_lons) / len(all_lons)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add depot markers
    for d_id in params['set_D']:
        lon, lat = loc_data[d_id]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Depot {d_id}</b><br>Capacity: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home', prefix='fa')
        ).add_to(m)

    # Add client markers
    for c_id in params['set_C']:
        lon, lat = loc_data[c_id]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Client {c_id}</b><br>Demand: {params['client_demand'][c_id]}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # Add routes
    colors = ['darkblue', 'darkgreen', 'purple', 'orange', 'darkred', 'cadetblue']
    route_idx = 0

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) < 2:
            continue

        # Get coordinates for route
        coords = []
        for node in route:
            if node in loc_data:
                lon, lat = loc_data[node]
                coords.append([lat, lon])

        if len(coords) >= 2:
            color = colors[route_idx % len(colors)]

            # Add route line
            folium.PolyLine(
                coords,
                color=color,
                weight=4,
                opacity=0.8,
                popup=f"<b>Vehicle {v_id}</b><br>Route: {' -> '.join(str(n) for n in route)}<br>Load: {data['load']}<br>Distance: {data['distance']:.1f}km"
            ).add_to(m)

            # Add directional arrows
            for i in range(len(coords) - 1):
                mid_lat = (coords[i][0] + coords[i+1][0]) / 2
                mid_lon = (coords[i][1] + coords[i+1][1]) / 2
                folium.CircleMarker(
                    [mid_lat, mid_lon],
                    radius=3,
                    color=color,
                    fill=True,
                    popup=f"Vehicle {v_id}: {route[i]} → {route[i+1]}"
                ).add_to(m)

            route_idx += 1

    print(f"Generated map with {route_idx} routes")
    return m

def display_vehicle_summary(solution_details, params):
    """Display summary table."""
    if not solution_details:
        return

    print("\n--- Vehicle Summary ---")
    summary_data = []

    for v_id, data in solution_details['routes'].items():
        if not data['route']:
            continue

        route_str = ' -> '.join(str(n) for n in data['route'])
        depot_id = data['route'][0] if data['route'] else 'N/A'

        # Calculate demands satisfied
        demands = [str(params['client_demand'][c]) for c in data['clients_served']]
        demands_str = '-'.join(demands) if demands else 'None'

        summary_data.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': data['load'],
            'RouteSequence': route_str,
            'ClientsServed': len(data['clients_served']),
            'DemandsSatisfied': demands_str,
            'TotalDistance': round(data['distance'], 2),
            'TotalTime': round(data['distance'] * 2, 1),  # Estimate: 2 min/km
            'FuelCost': round(data['distance'] * params['total_unit_distance_cost'], 2)
        })

    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        print(summary_df.to_string(index=False))
    else:
        print("No active routes found.")

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("Starting VRP Case 2: Multiple Depots")

    depots_df = pd.read_csv('depots_case2.csv').head(3)  # Select first 3 depots
    clients_df = pd.read_csv('clients_case2.csv')
    vehicles_df = pd.read_csv('vehicles_case2.csv')

    print(f"Loaded: {str(len(depots_df))} depots, {str(len(clients_df))} clients, {str(len(vehicles_df))} vehicles")

    # Preprocess data
    print("\nPreprocessing data...")
    model_params = preprocess_data(depots_df, clients_df, vehicles_df, ORS_API_KEY)
    if model_params is None:
        print("Preprocessing failed.")
        exit()

    # Build and solve model
    print("\nBuilding model...")
    vrp_model = build_vrp_model(model_params)

    print("Solving model...")
    solver = SolverFactory('glpk')
    results = solver.solve(vrp_model, tee=True)

    # Extract and display results
    solution = extract_routes_and_details(vrp_model, results, model_params)
    if solution:
        # Create visualization
        map_viz = visualize_routes_folium(solution, model_params)
        if map_viz:
            display(map_viz)
            map_viz.save('case2_routes.html')
            print("\nMap saved as 'case2_routes.html'")

        # Display summary
        display_vehicle_summary(solution, model_params)

        print(f"\nTotal solution cost: {solution['total_cost']:.2f} COP")
    else:
        print("No valid solution found.")

"""# CASO 3"""

import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium
import time
import numpy as np

!apt-get install -y -qq coinor-cbc

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

# --- 1. Generación de Datos Realistas para Caso 3 ---
def generate_realistic_data_case3():
    """Genera datos realistas para 75 clientes y 5 depósitos en Bogotá"""

    # Definir zonas de Bogotá con sus límites geográficos
    bogota_zones = {
        'Norte': {'lat_min': 4.70, 'lat_max': 4.75, 'lon_min': -74.05, 'lon_max': -74.00},
        'Sur': {'lat_min': 4.55, 'lat_max': 4.60, 'lon_min': -74.15, 'lon_max': -74.10},
        'Este': {'lat_min': 4.60, 'lat_max': 4.65, 'lon_min': -74.00, 'lon_max': -73.95},
        'Oeste': {'lat_min': 4.65, 'lat_max': 4.70, 'lon_min': -74.20, 'lon_max': -74.15},
        'Centro': {'lat_min': 4.60, 'lat_max': 4.65, 'lon_min': -74.10, 'lon_max': -74.05}
    }

    # Generar 5 depósitos estratégicamente ubicados
    depots_data = {
        'DepotID': [],
        'Longitude': [],
        'Latitude': [],
        'Capacity': []
    }

    for i, (zone_name, zone_coords) in enumerate(bogota_zones.items()):
        depots_data['DepotID'].append(f'D{i+1}_{zone_name}')
        depots_data['Longitude'].append(
            random.uniform(zone_coords['lon_min'], zone_coords['lon_max'])
        )
        depots_data['Latitude'].append(
            random.uniform(zone_coords['lat_min'], zone_coords['lat_max'])
        )
        # Capacidades mayores para manejar más clientes
        depots_data['Capacity'].append(random.randint(3000, 5000))

    depots_df = pd.DataFrame(depots_data)

    # Generar 75 clientes distribuidos por toda Bogotá
    clients_data = {
        'ClientID': [],
        'Demand': [],
        'Longitude': [],
        'Latitude': [],
        'Zone': []
    }

    num_clients = 50
    clients_per_zone = num_clients // len(bogota_zones)
    remaining = num_clients % len(bogota_zones)

    client_id = 1
    for zone_idx, (zone_name, zone_coords) in enumerate(bogota_zones.items()):
        clients_in_zone = clients_per_zone + (1 if zone_idx < remaining else 0)

        for _ in range(clients_in_zone):
            clients_data['ClientID'].append(f'C{client_id}')
            # Demandas variadas para simular diferentes tipos de clientes
            demand_type = random.choice(['small', 'medium', 'large'])
            if demand_type == 'small':
                demand = random.randint(10, 30)
            elif demand_type == 'medium':
                demand = random.randint(31, 60)
            else:
                demand = random.randint(61, 100)

            clients_data['Demand'].append(demand)
            clients_data['Longitude'].append(
                random.uniform(zone_coords['lon_min'], zone_coords['lon_max'])
            )
            clients_data['Latitude'].append(
                random.uniform(zone_coords['lat_min'], zone_coords['lat_max'])
            )
            clients_data['Zone'].append(zone_name)
            client_id += 1

    clients_df = pd.DataFrame(clients_data)

    # Generar flota heterogénea de vehículos (15 vehículos)
    vehicles_data = {
        'VehicleID': [],
        'Capacity': [],
        'Range': [],
        'Type': []
    }

    vehicle_types = [
        {'type': 'Small', 'capacity': (150, 200), 'range': (300, 400), 'count': 8},
        {'type': 'Medium', 'capacity': (201, 300), 'range': (401, 500), 'count': 10},
        {'type': 'Large', 'capacity': (301, 400), 'range': (501, 600), 'count': 7}
    ]

    v_id = 1
    for vtype in vehicle_types:
        for _ in range(vtype['count']):
            vehicles_data['VehicleID'].append(f'V{v_id}')
            vehicles_data['Capacity'].append(
                random.randint(vtype['capacity'][0], vtype['capacity'][1])
            )
            vehicles_data['Range'].append(
                random.randint(vtype['range'][0], vtype['range'][1])
            )
            vehicles_data['Type'].append(vtype['type'])
            v_id += 1

    vehicles_df = pd.DataFrame(vehicles_data)

    # Guardar los archivos
    depots_df.to_csv('depots_case3.csv', index=False)
    clients_df.to_csv('clients_case3.csv', index=False)
    vehicles_df.to_csv('vehicles_case3.csv', index=False)

    print(f"Caso 3 generado: {len(depots_df)} depósitos, {len(clients_df)} clientes, {len(vehicles_df)} vehículos")

    return depots_df, clients_df, vehicles_df

# --- 2. Función optimizada para obtener matrices de distancia ---
def get_distance_matrix_batch(locations_dict, node_ids, api_key, batch_size=40):
    """Obtiene matriz de distancias en lotes para manejar límites de API"""
    n = len(node_ids)
    distances = {}

    # Procesar en lotes para evitar límites de API
    for i in range(0, n, batch_size):
        batch_end = min(i + batch_size, n)
        batch_nodes = node_ids[i:batch_end]

        # Llamar a la API para este lote
        api_locations = [[locations_dict[node_id][0], locations_dict[node_id][1]]
                        for node_id in batch_nodes]

        body = {"locations": api_locations, "metrics": ["distance"]}
        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': api_key,
            'Content-Type': 'application/json; charset=utf-8'
        }

        response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)

        if response.status_code == 200:
            data = response.json()
            distance_matrix = data.get("distances")

            if distance_matrix:
                for idx_i, node_i in enumerate(batch_nodes):
                    for idx_j, node_j in enumerate(batch_nodes):
                        dist = distance_matrix[idx_i][idx_j] if distance_matrix[idx_i][idx_j] is not None else float('inf')
                        distances[(node_i, node_j)] = dist / 1000.0  # Convertir a km

        # Pequeña pausa para no sobrecargar la API
        time.sleep(0.5)

    return distances

# --- 3. Preprocesamiento mejorado para Caso 3 ---
def preprocess_data_case3(depots_df, clients_df, vehicles_df, api_key):
    """Preprocesamiento optimizado para escenario grande"""
    params = {}

    # Crear diccionario de ubicaciones
    locations = {}
    for _, row in depots_df.iterrows():
        locations[row['DepotID']] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations[row['ClientID']] = (row['Longitude'], row['Latitude'])

    params['locations'] = locations

    # Nodos
    depot_ids = depots_df['DepotID'].tolist()
    client_ids = clients_df['ClientID'].tolist()
    all_nodes = depot_ids + client_ids

    print(f"Procesando distancias para {len(all_nodes)} nodos...")

    # Obtener distancias por lotes
    distances = {}

    # 1. Distancias depot-cliente (más importantes)
    print("Calculando distancias depot-cliente...")
    for d_id in depot_ids:
        for c_id in client_ids:
            # Usar distancia euclidiana ajustada para Bogotá
            d_loc = locations[d_id]
            c_loc = locations[c_id]
            dist = calculate_bogota_distance(d_loc[1], d_loc[0], c_loc[1], c_loc[0])
            distances[(d_id, c_id)] = dist

    # 2. Distancias cliente-cliente (solo las necesarias)
    print("Calculando distancias cliente-cliente...")
    # Solo calcular para clientes cercanos para reducir complejidad
    for i, c1_id in enumerate(client_ids):
        for j, c2_id in enumerate(client_ids):
            if i != j:
                c1_loc = locations[c1_id]
                c2_loc = locations[c2_id]
                dist = calculate_bogota_distance(c1_loc[1], c1_loc[0], c2_loc[1], c2_loc[0])
                # Solo guardar si están relativamente cerca (menos de 10 km)
                if dist < 10:
                    distances[(c1_id, c2_id)] = dist

    # 3. Distancias cliente-depot (para retornos)
    print("Calculando distancias cliente-depot...")
    for c_id in client_ids:
        for d_id in depot_ids:
            c_loc = locations[c_id]
            d_loc = locations[d_id]
            dist = calculate_bogota_distance(c_loc[1], c_loc[0], d_loc[1], d_loc[0])
            distances[(c_id, d_id)] = dist

    params['distances'] = distances

    # Otros parámetros
    params['depot_capacity'] = dict(zip(depots_df['DepotID'], depots_df['Capacity']))
    params['client_demand'] = dict(zip(clients_df['ClientID'], clients_df['Demand']))
    params['vehicle_capacity'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity']))
    params['vehicle_range'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Range']))
    params['vehicle_type'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Type']))

    # Costos
    params['F_freight_cost'] = 5000
    params['M_maintenance_cost'] = 23
    params['Pf_fuel_cost'] = 525.1
    params['total_unit_distance_cost'] = params['F_freight_cost'] + params['M_maintenance_cost'] + params['Pf_fuel_cost']

    # Conjuntos
    params['set_D'] = depot_ids
    params['set_C'] = client_ids
    params['set_V'] = vehicles_df['VehicleID'].tolist()
    params['set_N'] = all_nodes

    print(f"Preprocesamiento completado: {len(distances)} distancias calculadas")

    return params

# --- 4. Función auxiliar para calcular distancia en Bogotá ---
def calculate_bogota_distance(lat1, lon1, lat2, lon2):
    """Calcula distancia ajustada para Bogotá con factor de corrección urbano"""
    # Distancia euclidiana con factor de corrección para calles urbanas
    urban_factor = 1.3  # Las rutas urbanas son ~30% más largas que línea recta

    # Conversión aproximada para Bogotá
    lat_km = 111.32  # km por grado de latitud
    lon_km = 111.32 * math.cos(math.radians(4.6))  # km por grado de longitud en Bogotá

    dlat = (lat2 - lat1) * lat_km
    dlon = (lon2 - lon1) * lon_km

    euclidean_dist = math.sqrt(dlat**2 + dlon**2)
    return euclidean_dist * urban_factor

# --- 5. Modelo VRP optimizado para Caso 3 ---
def build_vrp_model_case3(params):
    """Modelo VRP con optimizaciones para escala grande"""
    model = pyo.ConcreteModel(name="VRP_Case3_Realistic")

    # --- CONJUNTOS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])

    # --- PARÁMETROS ---
    # Inicializar distancias con valor por defecto alto para pares no calculados
    def dist_init(model, i, j):
        if (i, j) in params['distances']:
            return params['distances'][(i, j)]
        elif i == j:
            return 0
        else:
            return 9999  # Distancia muy alta para evitar estas rutas

    model.dist = pyo.Param(model.N, model.N, initialize=dist_init)
    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.cost_per_km = pyo.Param(initialize=params['total_unit_distance_cost'])
    model.num_clients = pyo.Param(initialize=len(params['set_C']))

    # --- VARIABLES DE DECISIÓN ---
    # Variable binaria: asignación de cliente a vehículo
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)

    # Variables de ruteo
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary)
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)

    # Variables auxiliares para subtour elimination
    model.u_jk = pyo.Var(model.C, model.V, bounds=(0, model.num_clients))

    # --- FUNCIÓN OBJETIVO ---
    def objective_rule(m):
        cost = 0
        # Costo de rutas depot-cliente
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k]
                   for d in m.D for j in m.C for k in m.V
                   if m.dist[d,j] < 9999)
        # Costo de rutas cliente-cliente
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k]
                   for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V
                   if m.dist[c1,c2] < 9999)
        # Costo de retorno a depot
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k]
                   for c in m.C for d in m.D for k in m.V
                   if m.dist[c,d] < 9999)
        return cost

    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- RESTRICCIONES ---

    # 1. Cada cliente debe ser visitado exactamente una vez
    def client_assignment_rule(m, j):
        return sum(m.Y_jk[j,k] for k in m.V) == 1
    model.client_assignment_con = pyo.Constraint(model.C, rule=client_assignment_rule)

    # 2. Capacidad del vehículo
    def vehicle_capacity_rule(m, k):
        return sum(m.client_demand[j] * m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    model.vehicle_capacity_con = pyo.Constraint(model.V, rule=vehicle_capacity_rule)

    # 3. Un vehículo inicia desde máximo un depot
    def vehicle_start_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) <= 1
    model.vehicle_start_con = pyo.Constraint(model.V, rule=vehicle_start_rule)

    # 4. Si un vehículo sirve clientes, debe iniciar desde un depot
    def link_Y_to_start_rule(m, k):
        return sum(m.Y_jk[j,k] for j in m.C) <= m.num_clients * sum(m.z_djk[d,j,k] for d in m.D for j in m.C)
    model.link_Y_to_start_con = pyo.Constraint(model.V, rule=link_Y_to_start_rule)

    # 5. Conservación de flujo
    def flow_conservation_rule(m, j, k):
        inflow = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j and m.dist[c1,j] < 9999)
        outflow = sum(m.x_prime_cdk[j,d,k] for d in m.D) + sum(m.x_c1c2k[j,c2,k] for c2 in m.C if j != c2 and m.dist[j,c2] < 9999)
        return inflow == outflow
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_conservation_rule)

    # 6. Enlace entre asignación y visita
    def link_assign_to_visit_rule(m, j, k):
        inflow = sum(m.z_djk[d,j,k] for d in m.D) + sum(m.x_c1c2k[c1,j,k] for c1 in m.C if c1 != j and m.dist[c1,j] < 9999)
        return inflow == m.Y_jk[j,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_to_visit_rule)

    # 7. Capacidad del depot
    def depot_capacity_rule(m, d):
        return sum(m.client_demand[j] * m.z_djk[d,j,k] for j in m.C for k in m.V) <= m.depot_cap[d]
    model.depot_capacity_con = pyo.Constraint(model.D, rule=depot_capacity_rule)

    # 8. Rango del vehículo (simplificado para velocidad)
    def vehicle_range_rule(m, k):
        total_dist = 0
        total_dist += sum(m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C if m.dist[d,j] < 9999)
        total_dist += sum(m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2 and m.dist[c1,c2] < 9999)
        total_dist += sum(m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D if m.dist[c,d] < 9999)
        return total_dist <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 9. No self-loops
    def no_self_loops_rule(m, c, k):
        return m.x_c1c2k[c,c,k] == 0
    model.no_self_loops_con = pyo.Constraint(model.C, model.V, rule=no_self_loops_rule)

    # 10. Si un vehículo termina en un depot, debe haber iniciado
    def end_tour_rule(m, k):
        starts = sum(m.z_djk[d,j,k] for d in m.D for j in m.C)
        ends = sum(m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return starts == ends
    model.end_tour_con = pyo.Constraint(model.V, rule=end_tour_rule)

    # 11. Subtour elimination simplificado
    def subtour_bounds_rule(m, j, k):
        return m.u_jk[j,k] >= m.Y_jk[j,k]
    model.subtour_bounds_con = pyo.Constraint(model.C, model.V, rule=subtour_bounds_rule)

    def subtour_order_rule(m, c1, c2, k):
        if c1 != c2 and m.dist[c1,c2] < 9999:
            return m.u_jk[c1,k] - m.u_jk[c2,k] + m.num_clients * m.x_c1c2k[c1,c2,k] <= m.num_clients - 1
        else:
            return pyo.Constraint.Skip
    model.subtour_order_con = pyo.Constraint(model.C, model.C, model.V, rule=subtour_order_rule)

    return model

# --- 6. Resolver con estrategia optimizada ---
def solve_model_case3(model, time_limit=600):
    """Resuelve el modelo con parámetros optimizados para caso grande"""
    # Intentar usar diferentes solvers según disponibilidad
    solver = None
    for solver_name in ['cbc', 'glpk', 'highs']:
        try:
            solver = SolverFactory(solver_name)
            if solver.available():
                print(f"Usando solver: {solver_name}")
                break
        except:
            continue

    if solver is None:
        print("ERROR: No se encontró ningún solver disponible")
        return None

    print(f"Resolviendo modelo con límite de tiempo de {time_limit} segundos...")
    print("Esto puede tomar varios minutos para 75 clientes...")

    try:
        results = solver.solve(model, tee=True)
    except Exception as e:
        print(f"Error al resolver el modelo: {e}")
        return None

    return results

# --- 7. Extraer y visualizar resultados ---
def extract_routes_case3(model, results, params):
    """Extrae las rutas de la solución"""
    if results is None:
        print("No se recibieron resultados del solver")
        return None

    if results.solver.termination_condition not in [pyo.TerminationCondition.optimal,
                                                    pyo.TerminationCondition.feasible,
                                                    pyo.TerminationCondition.maxTimeLimit]:
        print(f"No se encontró solución factible. Estado: {results.solver.termination_condition}")
        return None

    print("\n--- Solución encontrada ---")
    try:
        total_cost = pyo.value(model.total_cost)
        print(f"Costo total: ${total_cost:,.2f} COP")
    except:
        total_cost = 0
        print("Advertencia: No se pudo obtener el valor de la función objetivo")

    solution_details = {'total_cost': total_cost, 'routes': {}}

    # Extraer rutas por vehículo
    for k_id in params['set_V']:
        vehicle_route = []
        vehicle_load = 0
        vehicle_distance = 0
        clients_served = []

        # Encontrar inicio desde depot
        start_depot = None
        first_client = None

        for d_id in params['set_D']:
            for c_id in params['set_C']:
                if pyo.value(model.z_djk[d_id, c_id, k_id]) and pyo.value(model.z_djk[d_id, c_id, k_id]) > 0.5:
                    start_depot = d_id
                    first_client = c_id
                    vehicle_route = [d_id, c_id]
                    if (d_id, c_id) in params['distances']:
                        vehicle_distance += params['distances'][(d_id, c_id)]
                    break
            if start_depot:
                break

        if not start_depot:
            # Vehículo no usado
            solution_details['routes'][k_id] = {
                'route': [],
                'load': 0,
                'distance': 0,
                'clients_served': [],
                'vehicle_type': params['vehicle_type'][k_id]
            }
            continue

        # Seguir la ruta
        current = first_client
        visited = {first_client}
        safety_counter = 0
        max_iterations = len(params['set_C'])

        while current and safety_counter < max_iterations:
            next_client = None

            # Buscar siguiente cliente
            for c_id in params['set_C']:
                if c_id != current and c_id not in visited:
                    if (current, c_id) in params['distances'] and params['distances'][(current, c_id)] < 9999:
                        if pyo.value(model.x_c1c2k[current, c_id, k_id]) and pyo.value(model.x_c1c2k[current, c_id, k_id]) > 0.5:
                            next_client = c_id
                            break

            if next_client:
                vehicle_route.append(next_client)
                visited.add(next_client)
                if (current, next_client) in params['distances']:
                    vehicle_distance += params['distances'][(current, next_client)]
                current = next_client
            else:
                # Buscar retorno a depot
                for d_id in params['set_D']:
                    if pyo.value(model.x_prime_cdk[current, d_id, k_id]) and pyo.value(model.x_prime_cdk[current, d_id, k_id]) > 0.5:
                        vehicle_route.append(d_id)
                        if (current, d_id) in params['distances']:
                            vehicle_distance += params['distances'][(current, d_id)]
                        break
                break

            safety_counter += 1

        # Calcular clientes servidos y carga
        for c_id in params['set_C']:
            if pyo.value(model.Y_jk[c_id, k_id]) and pyo.value(model.Y_jk[c_id, k_id]) > 0.5:
                clients_served.append(c_id)
                vehicle_load += params['client_demand'][c_id]

        solution_details['routes'][k_id] = {
            'route': vehicle_route,
            'load': vehicle_load,
            'distance': vehicle_distance,
            'clients_served': clients_served,
            'vehicle_type': params['vehicle_type'][k_id],
            'capacity': params['vehicle_capacity'][k_id],
            'range': params['vehicle_range'][k_id]
        }

    return solution_details

# --- 8. Visualización mejorada para Caso 3 ---
def visualize_routes_case3_folium(solution_details, params):
    """Crea mapa interactivo con todas las rutas"""
    if not solution_details:
        print("No hay solución para visualizar")
        return None

    # Centro del mapa en Bogotá
    map_center = [4.65, -74.05]
    m = folium.Map(location=map_center, zoom_start=11)

    # Colores por tipo de vehículo
    vehicle_colors = {
        'Small': 'lightblue',
        'Medium': 'orange',
        'Large': 'darkred'
    }

    # Agregar depósitos
    for d_id in params['set_D']:
        lon, lat = params['locations'][d_id]
        folium.Marker(
            [lat, lon],
            popup=f"<b>{d_id}</b><br>Capacidad: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home', prefix='fa')
        ).add_to(m)

    # Agregar clientes coloreados por si fueron servidos
    served_clients = set()
    for v_data in solution_details['routes'].values():
        served_clients.update(v_data['clients_served'])

    for c_id in params['set_C']:
        lon, lat = params['locations'][c_id]
        color = 'green' if c_id in served_clients else 'gray'
        icon_type = 'check' if c_id in served_clients else 'times'

        folium.Marker(
            [lat, lon],
            popup=f"<b>{c_id}</b><br>Demanda: {params['client_demand'][c_id]}<br>{'Servido' if c_id in served_clients else 'No servido'}",
            icon=folium.Icon(color=color, icon=icon_type)
        ).add_to(m)

    # Agregar rutas
    route_count = 0
    for v_id, v_data in solution_details['routes'].items():
        if len(v_data['route']) > 1:
            route_count += 1
            route_coords = []

            for node in v_data['route']:
                if node in params['locations']:
                    lon, lat = params['locations'][node]
                    route_coords.append([lat, lon])

            if len(route_coords) > 1:
                v_type = v_data['vehicle_type']
                color = vehicle_colors.get(v_type, 'blue')

                folium.PolyLine(
                    route_coords,
                    color=color,
                    weight=3,
                    opacity=0.7,
                    popup=f"<b>{v_id} ({v_type})</b><br>"
                          f"Ruta: {' → '.join(v_data['route'])}<br>"
                          f"Carga: {v_data['load']}/{v_data['capacity']}<br>"
                          f"Distancia: {v_data['distance']:.1f}/{v_data['range']} km<br>"
                          f"Clientes: {len(v_data['clients_served'])}"
                ).add_to(m)

    # Agregar leyenda
    legend_html = '''
    <div style="position: fixed;
                top: 50px; right: 50px; width: 200px; height: 200px;
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; border-radius: 5px; padding: 10px">
    <p style="margin: 0;"><b>Leyenda</b></p>
    <p style="margin: 5px;"><i class="fa fa-home" style="color:red"></i> Depósito</p>
    <p style="margin: 5px;"><i class="fa fa-check" style="color:green"></i> Cliente servido</p>
    <p style="margin: 5px;"><i class="fa fa-times" style="color:gray"></i> Cliente no servido</p>
    <p style="margin: 5px;"><b>Tipos de vehículo:</b></p>
    <p style="margin: 5px;"><span style="color:lightblue">━━</span> Pequeño</p>
    <p style="margin: 5px;"><span style="color:orange">━━</span> Mediano</p>
    <p style="margin: 5px;"><span style="color:darkred">━━</span> Grande</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    print(f"\nMapa generado con {route_count} rutas activas")
    return m

# --- 9. Análisis de resultados y métricas ---
def analyze_solution_case3(solution_details, params):
    """Genera análisis detallado de la solución"""
    if not solution_details:
        return None

    print("\n=== ANÁLISIS DETALLADO DE LA SOLUCIÓN ===\n")

    # Métricas generales
    total_clients = len(params['set_C'])
    served_clients = set()
    total_distance = 0
    total_load = 0
    vehicles_used = 0

    # Estadísticas por tipo de vehículo
    stats_by_type = {
        'Small': {'count': 0, 'distance': 0, 'load': 0, 'clients': 0},
        'Medium': {'count': 0, 'distance': 0, 'load': 0, 'clients': 0},
        'Large': {'count': 0, 'distance': 0, 'load': 0, 'clients': 0}
    }

    # Procesar cada ruta
    for v_id, v_data in solution_details['routes'].items():
        if v_data['route']:
            vehicles_used += 1
            served_clients.update(v_data['clients_served'])
            total_distance += v_data['distance']
            total_load += v_data['load']

            v_type = v_data['vehicle_type']
            stats_by_type[v_type]['count'] += 1
            stats_by_type[v_type]['distance'] += v_data['distance']
            stats_by_type[v_type]['load'] += v_data['load']
            stats_by_type[v_type]['clients'] += len(v_data['clients_served'])

    # Imprimir resultados
    print(f"📊 MÉTRICAS GENERALES:")
    print(f"   - Costo total: ${solution_details['total_cost']:,.2f} COP")
    print(f"   - Clientes totales: {total_clients}")
    print(f"   - Clientes servidos: {len(served_clients)} ({len(served_clients)/total_clients*100:.1f}%)")
    print(f"   - Vehículos utilizados: {vehicles_used} de {len(params['set_V'])}")
    print(f"   - Distancia total: {total_distance:.1f} km")
    print(f"   - Carga total entregada: {total_load} unidades")

    print(f"\n📈 ESTADÍSTICAS POR TIPO DE VEHÍCULO:")
    for v_type, stats in stats_by_type.items():
        if stats['count'] > 0:
            print(f"\n   {v_type}:")
            print(f"   - Vehículos usados: {stats['count']}")
            print(f"   - Distancia promedio: {stats['distance']/stats['count']:.1f} km")
            print(f"   - Carga promedio: {stats['load']/stats['count']:.1f} unidades")
            print(f"   - Clientes por vehículo: {stats['clients']/stats['count']:.1f}")

    # Análisis de eficiencia
    print(f"\n🎯 INDICADORES DE EFICIENCIA:")
    if vehicles_used > 0:
        print(f"   - Distancia promedio por vehículo: {total_distance/vehicles_used:.1f} km")
        print(f"   - Carga promedio por vehículo: {total_load/vehicles_used:.1f} unidades")
        print(f"   - Costo promedio por cliente servido: ${solution_details['total_cost']/len(served_clients):,.2f} COP")
        print(f"   - Costo por km: ${solution_details['total_cost']/total_distance:,.2f} COP/km")

    # Análisis por depósito
    print(f"\n🏭 UTILIZACIÓN POR DEPÓSITO:")
    depot_usage = {d: 0 for d in params['set_D']}
    for v_data in solution_details['routes'].values():
        if v_data['route']:
            start_depot = v_data['route'][0]
            if start_depot in params['set_D']:
                depot_usage[start_depot] += 1

    for depot, count in depot_usage.items():
        print(f"   - {depot}: {count} vehículos")

    return {
        'total_clients': total_clients,
        'served_clients': len(served_clients),
        'service_rate': len(served_clients)/total_clients*100,
        'vehicles_used': vehicles_used,
        'total_distance': total_distance,
        'total_cost': solution_details['total_cost'],
        'stats_by_type': stats_by_type
    }

# --- 10. Análisis de sensibilidad ---
def sensitivity_analysis_case3(base_params, base_solution):
    """Realiza análisis de sensibilidad sobre parámetros clave"""
    print("\n=== ANÁLISIS DE SENSIBILIDAD ===\n")

    sensitivity_results = {}

    # 1. Variación en costos de combustible (±20%)
    print("1. Analizando impacto de variación en costos de combustible...")

    fuel_variations = [0.8, 1.0, 1.2]  # -20%, base, +20%
    fuel_results = []

    for factor in fuel_variations:
        params_copy = base_params.copy()
        params_copy['Pf_fuel_cost'] = base_params['Pf_fuel_cost'] * factor
        params_copy['total_unit_distance_cost'] = (
            params_copy['F_freight_cost'] +
            params_copy['M_maintenance_cost'] +
            params_copy['Pf_fuel_cost']
        )

        # Calcular nuevo costo con el mismo ruteo
        new_cost = 0
        for v_data in base_solution['routes'].values():
            if v_data['distance'] > 0:
                new_cost += v_data['distance'] * params_copy['total_unit_distance_cost']

        fuel_results.append({
            'factor': factor,
            'fuel_cost': params_copy['Pf_fuel_cost'],
            'total_cost': new_cost,
            'cost_change': ((new_cost - base_solution['total_cost']) / base_solution['total_cost']) * 100
        })

    sensitivity_results['fuel_cost'] = fuel_results

    # Imprimir resultados
    print("\n   Resultados - Variación en costo de combustible:")
    for result in fuel_results:
        print(f"   Factor {result['factor']:.1f}x: Costo = ${result['total_cost']:,.0f} ({result['cost_change']:+.1f}%)")

    # 2. Análisis de capacidad de depósitos
    print("\n2. Analizando utilización de capacidad de depósitos...")

    depot_utilization = {}
    for d_id in base_params['set_D']:
        total_demand_from_depot = 0
        for v_id, v_data in base_solution['routes'].items():
            if v_data['route'] and v_data['route'][0] == d_id:
                total_demand_from_depot += v_data['load']

        utilization = (total_demand_from_depot / base_params['depot_capacity'][d_id]) * 100
        depot_utilization[d_id] = {
            'capacity': base_params['depot_capacity'][d_id],
            'used': total_demand_from_depot,
            'utilization': utilization
        }

    print("\n   Utilización de depósitos:")
    for d_id, util in depot_utilization.items():
        print(f"   {d_id}: {util['used']}/{util['capacity']} ({util['utilization']:.1f}%)")

    sensitivity_results['depot_utilization'] = depot_utilization

    return sensitivity_results

# --- 11. Generar archivo de verificación ---
def save_verification_case3(solution_details, params):
    """Genera archivo de verificación para Caso 3"""
    verification_data = []

    for v_id, v_data in solution_details['routes'].items():
        if not v_data['route']:
            continue

        # Extraer información
        route_str = '-'.join(v_data['route'])
        depot_id = v_data['route'][0] if v_data['route'] else 'N/A'

        # Calcular demandas satisfechas
        demands_satisfied = []
        for node in v_data['route']:
            if node in params['client_demand']:
                demands_satisfied.append(str(params['client_demand'][node]))
        demands_str = '-'.join(demands_satisfied) if demands_satisfied else '0'

        # Calcular tiempo estimado (2 min/km)
        time_minutes = v_data['distance'] * 2

        # Calcular costo de combustible
        fuel_cost = v_data['distance'] * params['total_unit_distance_cost']

        verification_data.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': v_data['load'],
            'RouteSequence': route_str,
            'ClientsServed': len(v_data['clients_served']),
            'DemandsSatisfied': demands_str,
            'TotalDistance': round(v_data['distance'], 2),
            'TotalTime': round(time_minutes, 1),
            'FuelCost': round(fuel_cost, 2)
        })

    # Crear DataFrame y guardar
    verification_df = pd.DataFrame(verification_data)

    if not verification_df.empty:
        # Ordenar columnas según especificación
        column_order = [
            'VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence',
            'ClientsServed', 'DemandsSatisfied', 'TotalDistance',
            'TotalTime', 'FuelCost'
        ]
        verification_df = verification_df[column_order]

        # Guardar archivo
        filename = 'verificacion_caso3.csv'
        verification_df.to_csv(filename, index=False)
        print(f"\n✅ Archivo de verificación guardado: {filename}")
        print(f"   - Vehículos con rutas: {len(verification_df)}")
        print(f"   - Total clientes servidos: {verification_df['ClientsServed'].sum()}")

    return verification_df

# --- 12. Función principal para ejecutar Caso 3 ---
def run_case3():
    """Ejecuta el Caso 3 completo"""
    print("="*60)
    print("CASO 3: ESCENARIO REALISTA CON 75 CLIENTES")
    print("="*60)

    # 1. Generar o cargar datos
    print("\n1. Generando datos del escenario...")
    depots_df, clients_df, vehicles_df = generate_realistic_data_case3()

    # 2. Preprocesar datos
    print("\n2. Preprocesando datos...")
    params = preprocess_data_case3(depots_df, clients_df, vehicles_df, ORS_API_KEY)

    if not params:
        print("Error en preprocesamiento")
        return

    # 3. Construir modelo
    print("\n3. Construyendo modelo de optimización...")
    model = build_vrp_model_case3(params)

    # 4. Resolver modelo
    print("\n4. Resolviendo modelo...")
    results = solve_model_case3(model, time_limit=600)  # 10 minutos máximo

    if results is None:
        print("ERROR: No se pudo resolver el modelo")
        return None, None, None

    # 5. Extraer solución
    print("\n5. Extrayendo rutas de la solución...")
    solution = extract_routes_case3(model, results, params)

    if not solution:
        print("No se pudo extraer una solución válida")
        return None, None, None

    # 6. Analizar resultados
    print("\n6. Analizando resultados...")
    analysis = analyze_solution_case3(solution, params)

    # 7. Análisis de sensibilidad
    print("\n7. Realizando análisis de sensibilidad...")
    sensitivity = sensitivity_analysis_case3(params, solution)

    # 8. Visualizar en mapa
    print("\n8. Generando visualización...")
    folium_map = visualize_routes_case3_folium(solution, params)
    if folium_map:
        folium_map.save('caso3_rutas_mapa.html')
        print("   Mapa guardado como: caso3_rutas_mapa.html")
        display(folium_map)

    # 9. Generar archivo de verificación
    print("\n9. Generando archivo de verificación...")
    verification_df = save_verification_case3(solution, params)

    # 10. Generar gráficos adicionales
    print("\n10. Generando gráficos de análisis...")
    generate_analysis_plots_case3(solution, params, analysis, sensitivity)

    print("\n" + "="*60)
    print("CASO 3 COMPLETADO EXITOSAMENTE")
    print("="*60)

    return solution, analysis, sensitivity

# --- 13. Generar gráficos de análisis ---
def generate_analysis_plots_case3(solution, params, analysis, sensitivity):
    """Genera gráficos de análisis para el reporte"""

    # Configurar estilo
    plt.style.use('seaborn-v0_8-darkgrid')

    # 1. Gráfico de utilización de vehículos por tipo
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Subplot 1: Vehículos por tipo
    ax1 = axes[0, 0]
    vehicle_types = ['Small', 'Medium', 'Large']
    vehicles_used = [analysis['stats_by_type'][t]['count'] for t in vehicle_types]
    colors = ['lightblue', 'orange', 'darkred']

    bars1 = ax1.bar(vehicle_types, vehicles_used, color=colors)
    ax1.set_xlabel('Tipo de Vehículo')
    ax1.set_ylabel('Cantidad Utilizada')
    ax1.set_title('Utilización de Vehículos por Tipo')

    # Agregar valores en las barras
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom')

    # Subplot 2: Distancia promedio por tipo
    ax2 = axes[0, 1]
    avg_distances = []
    for t in vehicle_types:
        if analysis['stats_by_type'][t]['count'] > 0:
            avg_dist = analysis['stats_by_type'][t]['distance'] / analysis['stats_by_type'][t]['count']
            avg_distances.append(avg_dist)
        else:
            avg_distances.append(0)

    bars2 = ax2.bar(vehicle_types, avg_distances, color=colors)
    ax2.set_xlabel('Tipo de Vehículo')
    ax2.set_ylabel('Distancia Promedio (km)')
    ax2.set_title('Distancia Promedio Recorrida por Tipo de Vehículo')

    for bar, dist in zip(bars2, avg_distances):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                f'{dist:.1f}', ha='center', va='bottom')

    # Subplot 3: Análisis de sensibilidad - Costo de combustible
    ax3 = axes[1, 0]
    fuel_factors = [r['factor'] for r in sensitivity['fuel_cost']]
    fuel_costs = [r['total_cost'] for r in sensitivity['fuel_cost']]

    ax3.plot(fuel_factors, fuel_costs, 'o-', linewidth=2, markersize=8, color='green')
    ax3.set_xlabel('Factor de Costo de Combustible')
    ax3.set_ylabel('Costo Total (COP)')
    ax3.set_title('Sensibilidad al Costo de Combustible')
    ax3.grid(True, alpha=0.3)

    # Formatear eje Y con separadores de miles
    ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

    # Subplot 4: Utilización de depósitos
    ax4 = axes[1, 1]
    depot_names = list(sensitivity['depot_utilization'].keys())
    depot_utils = [sensitivity['depot_utilization'][d]['utilization'] for d in depot_names]

    bars4 = ax4.bar(depot_names, depot_utils, color='purple')
    ax4.set_xlabel('Depósito')
    ax4.set_ylabel('Utilización (%)')
    ax4.set_title('Utilización de Capacidad por Depósito')
    ax4.axhline(y=100, color='r', linestyle='--', alpha=0.5, label='Capacidad Máxima')

    for bar, util in zip(bars4, depot_utils):
        ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                f'{util:.1f}%', ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig('caso3_analisis_graficos.png', dpi=300, bbox_inches='tight')
    print("   Gráficos guardados como: caso3_analisis_graficos.png")
    plt.show()

    # 2. Gráfico de distribución de clientes servidos vs no servidos
    fig2, ax = plt.subplots(figsize=(8, 6))

    served = analysis['served_clients']
    not_served = analysis['total_clients'] - served

    sizes = [served, not_served]
    labels = [f'Servidos ({served})', f'No servidos ({not_served})']
    colors = ['#90EE90', '#FFB6C1']
    explode = (0.05, 0)

    ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
           shadow=True, startangle=90)
    ax.set_title('Cobertura de Servicio a Clientes')

    plt.savefig('caso3_cobertura_clientes.png', dpi=300, bbox_inches='tight')
    print("   Gráfico de cobertura guardado como: caso3_cobertura_clientes.png")
    plt.show()

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    print("Iniciando Caso 3 - Escenario Realista VRP")
    print("Este caso puede tomar varios minutos en resolver debido a la complejidad...")

    # Ejecutar el caso 3
    result = run_case3()
    if result is not None and all(r is not None for r in result):
        solution, analysis, sensitivity = result
        print("\n✅ Caso 3 completado exitosamente!")
        print("\nArchivos generados:")
        print("- verificacion_caso3.csv")
        print("- caso3_rutas_mapa.html")
        print("- caso3_analisis_graficos.png")
        print("- caso3_cobertura_clientes.png")
    else:
        print("\n❌ No se pudo completar el Caso 3.")
        print("El problema es demasiado restrictivo para los recursos disponibles.")
        print("Considera:")
        print("- Reducir el número de clientes")
        print("- Aumentar la capacidad de los vehículos")
        print("- Aumentar el número de vehículos disponibles")

    print("\n✅ Caso 3 completado exitosamente!")
    print("\nArchivos generados:")
    print("- verificacion_caso3.csv")
    print("- caso3_rutas_mapa.html")
    print("- caso3_analisis_graficos.png")
    print("- caso3_cobertura_clientes.png")

VRP Caso 3 › ejecución rápida (10 min máx.)

→ Solver highs | límite 600s | gap 5%
Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 37180 rows; 55775 cols; 292925 nonzeros; 53900 integer variables (53900 binary)
Coefficient ranges:
  Matrix [1e-01, 1e+02]
  Cost   [7e+02, 2e+05]
  Bound  [1e+00, 8e+01]
  RHS    [1e+00, 4e+03]
Presolving model
35305 rows, 53900 cols, 291050 nonzeros  0s
35305 rows, 53878 cols, 289665 nonzeros  5s

Solving MIP model with:
   35305 rows
   53878 cols (52003 binary, 0 integer, 0 implied int., 1875 continuous)
   289665 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dy

KeyboardInterrupt: 